<a href="https://colab.research.google.com/github/ImagingDataCommons/ai_medima_misc/blob/main/nnunet/notebooks/idc_nlst_nnunet_and_bpr_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Insert description of what this notebook does!

This notebook performs inference for both nnUNet and BPR for the NLST collection. The notebook should be run using the GCE VM `colab-test-vm-1-vm`

The advantage of combining the analysis for the two use cases are that the download of the DICOM files and the conversion of DICOM to nifti is only performed once. 

Notes to do 8-22-22 week
1. Davide modified the OHIF viewer to work with multiple SR - deploy OHIF again 
2. Make sure 2 SR and SEG etc look fine in OHIF 
3. Make sure no strange characters/output in SRs 
4. Add series description to BPR SRs 
5. Once the above is done, run over entire NLST cohort  

Notes to do 8-15-22 week 
1. Put query for NSCLC-Radiomics in repo - done 
2. Recheck the nnUNet features SR - weird characters, check with DICOM Validator - dciodvfy (dclunie.com) - done
3. Recheck the other BPR SR regions and BPR SR landmarks for weird characters - done
4. Since we have a problem of viewing both the BPR regions SR and BPR landmarks SR, temporarily create two dicom stores - done
5. Run 50 series and check output and cost - done
6. Set up the cost estimation project/VM?? 
7. Run 50 series on the cost estimation VM and make sure everything works 

Notes to do 8-8-22 week
1. Use csv for the BPR regions code mapping - done
2. Put query in repo for NLST - done 
3. After Andrey reviews notebook - run on set of 50 series from NLST 
4. Once set, move functions to git repo https://github.com/ImagingDataCommons/ai_medima_misc
5. (Change to use subprocess commands?)
6. (still need to resolve OHIF issue of viewing 2 SR) 

Notes to do 8-1-22 week:
1. Add in SR creation for BPR landmarks - done
2. Add in SR creation for nnUNet 3D shape features - done
3. Make sure notebook can be used for both GCE VM and colab VM - add in auth and location of s5cmd - working
4. Move order of cells for pyplastimatch - done
5. Log errors for dcm2nii if creates no volumes or more than 1 volume - done
6. NLST Query - for cancer positive patients also with filtering for missing slices - done
7. NSCLC-Radiomics Query - for detecting patients with missing/repeated slices - done 

Notes to do 7-18-22 week:
0. Clean up the parameterization, environment etc. - done
1. Remove the 2D model computation - done
2. Add in the OHIF viewer instructions - done
3. Add in the creation of the SR for BPR - done
4. (Can I run without resampling?) 
4. Add in the OHIF viewer functions - done
5. Insert flags for each step that is performed, at end insert these values into a table -- discuss with Dennis
5. Redo the creation of the BQ tables for inference time, total time, saving stages that were complete. -- check the notebook from Dennis and discuss - done for now.
6. Use dcm2niix for DICOM to nifti instead of plastimatch -- if multiple files are created this will help us to identify cases we cannot process. - done 
7. Add in the computation of shape features (only 3D?) - done
8. Create SR for above instead of csv - working 
9. Dennis to move some functions to the git repo 
10. Query for nsclc-radiomics to figure out patients that have missing slices
8. Add in instructions for setting up GCE VM

Notes to do misc:
1. Need to move functions to git repo!
2. Remove the DICOM sorting step. 
3. (Dennis - in nrrd_to_dicomseg the processed_nrrd_path doesn't exist)


**Deploying your own OHIF viewer instance**

In order to view the results, we will use the output json files created from the body part regression and create a structured report. This will show a label for each slice as an annotation in the custom OHIF viewer. 

Use these instructions to first deploy your own OHIF instance: 

https://tinyurl.com/idc-ohif-gcp 

# **Parameterization**

We have set up the GCE VM to use the NVIDIA Tesla V100 GPU. 

In [ ]:
!nvidia-smi

Thu Aug 25 22:37:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
gpu_type = !nvidia-smi -L
print(gpu_type[0])
# If using GCE VM 
if ('Tesla V100' in gpu_type[0]):
  gce_vm = 1
# Else using Colab VM
else:
  gce_vm = 0

GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-61713c39-2805-1328-f2b9-dec4fe407da3)


Set up the proper credentials in order to use for instance, the python API 
for BigQuery. Credentials will be different depending on if you use the GCE VM
or the Colab VM. 

In [ ]:
import os 

# If using GCE VM get credentials 
if (gce_vm):
  os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/SA_key_018.json"
  s5cmd_path = 's5cmd'

# If using Colab VM authorize to get credentials 
# Set up credentials needed for s5cmd 
else:
  # Authorize 
  from google.colab import auth
  auth.authenticate_user()
  import subprocess
  from google.colab import drive
  drive.mount('/content/gdrive')
  !mkdir -p ~/.aws
  !cp /content/gdrive/MyDrive/aws/credentials ~/.aws
  # Get s5cmd 
  !wget https://github.com/peak/s5cmd/releases/download/v2.0.0-beta/s5cmd_2.0.0-beta_Linux-64bit.tar.gz
  !tar zxf s5cmd_2.0.0-beta_Linux-64bit.tar.gz
  s5cmd_path = '/content/s5cmd'


In [ ]:
# SPECIFY: target table of results 

# Table and view names for holding cohort information 
project_name = 'idc-external-018'    
bucket_name = 'idc-medima-paper-dk'  
location_id = 'us-central1'
dataset_table_id = 'dataset_nlst' 
table_id_name = 'table_nlst' # holds the cohort information 
# table_view_id_name = 'nlst_pick_single_series_per_study_cancer_positive_filtering_missing_slices_axial'
# table_view_id_name = 'nlst_table_from_query_txt'
table_view_id_name = 'nlst_table'

# If set to 1, overwrite the table_view_id_name if it exists. It will use the query.txt file from the git repo. 
# If set to 0, do not overwrite the current table_view_id_name if it exists. 
# If it doesn't exist, the view will be created.
overwrite_view = 0

# Tables for inference and total times 
nnunet_table_id = 'nlst_nnunet_time_gcp_vm_test_all_run'
bpr_table_id = 'nlst_bpr_time_gcp_vm_test_all_run'     
nnunet_table_id_fullname = '.'.join([project_name, dataset_table_id, nnunet_table_id])
bpr_table_id_fullname = '.'.join([project_name, dataset_table_id, bpr_table_id])

# Bucket subdirectory to hold results 
# different bucket per run, for now. 
nlst_sub = 'nlst_gcp_vm_test_all_run'

# name of DICOM datastore 
# different datastore per run, for now. 
dataset_id = 'dataset_nlst'
datastore_id = 'datastore_nlst_gcp_vm_test_all_run'

# Choose 50 patients (=series) to analyze each time 
# Define the patient_offset to change the starting index. 
patient_offset = 0 # CHANGE THIS VALUE 


In [ ]:
# # SPECIFY: target table of results 

# # Table and view names for holding cohort information 
# project_name = 'idc-external-018'    
# bucket_name = 'idc-medima-paper-dk'  
# location_id = 'us-central1'
# dataset_table_id = 'dataset_nlst' 
# table_id_name = 'table_nlst' # holds the cohort information 
# # table_view_id_name = 'nlst_pick_single_series_per_study_cancer_positive_filtering_missing_slices_axial'
# # table_view_id_name = 'nlst_table_from_query_txt'
# table_view_id_name = 'nlst_table'

# # If set to 1, overwrite the table_view_id_name if it exists. It will use the query.txt file from the git repo. 
# # If set to 0, do not overwrite the current table_view_id_name if it exists. 
# # If it doesn't exist, the view will be created.
# overwrite_view = 0

# # Tables for inference and total times 
# # nnunet_table_id = 'nlst_nnunet_time_gcp_vm_no_resampling' 
# # bpr_table_id = 'nlst_bpr_time_gcp_vm_no_resampling' 
# nnunet_table_id = 'nlst_nnunet_time_gcp_vm_test_50_run'
# bpr_table_id = 'nlst_bpr_time_gcp_vm_test_50_run'     
# nnunet_table_id_fullname = '.'.join([project_name, dataset_table_id, nnunet_table_id])
# bpr_table_id_fullname = '.'.join([project_name, dataset_table_id, bpr_table_id])

# # Bucket subdirectory to hold results 
# # different bucket per run, for now. 
# # nlst_sub = 'nlst_50_series_gcp_vm_no_resampling' # CHANGE THIS to modify the sub directory for the bucket.
# nlst_sub = 'nlst_gcp_vm_test_50_run'

# # name of DICOM datastore 
# # different datastore per run, for now. 
# dataset_id = 'dataset_nlst'
# # datastore_id = 'datastore_nlst_50_series_gcp_vm_no_resampling' # CHANGE THIS to create a new datastore for these results.
# # datastore_id_with_bpr_regions = 'datastore_nlst_50_series_gcp_vm_no_resampling_with_bpr_regions'
# # datastore_id_with_bpr_landmarks = 'datastore_nlst_50_series_gcp_vm_no_resampling_with_bpr_landmarks'
# datastore_id_with_bpr_regions = 'datastore_nlst_gcp_vm_test_50_run_bpr_regions'
# datastore_id_with_bpr_landmarks = 'datastore_nlst_gcp_vm_test_50_run_bpr_landmarks'

# # Choose 50 patients (=series) to analyze each time 
# # Define the patient_offset to change the starting index. 
# patient_offset = 0 # CHANGE THIS VALUE 


Set the default region/zone for gcloud

In [ ]:
!gcloud config set project $project_name
!gcloud config set compute/region $location_id

# gcloud config set compute/region europe-west6
# gcloud config set compute/zone europe-west6-a

Updated property [core/project].
Updated property [compute/region].


Parameters for nnUNet

In [ ]:
nnunet_model = '3d_fullres' # choose from: "2d", "3d_lowres", "3d_fullres", "3d_cascade_fullres"
use_tta = True
export_prob_maps = False

As explained above,  you will have the option of being able to deploy your own instance of the OHIF viewer. Replace the name with your own in the variable `my_ohif_app`. 

In [ ]:
# my_ohif_app = None ### UNCOMMENT if you do not want to set up and use a viewer 
# my_ohif_app = 'idc-tester-dk' ### COMMENT out if you do not want to set up and use a viewer 
# my_ohif_app = 'idc2serversdeploy-3c769' # the two server solution -- we will need to have instructions to user on how to set this up. 
my_ohif_app = 'idc-tester-dk-2-server'

# **Environment Setup**

These packages need to be updated first.

In [ ]:
# %%capture

!pip uninstall -y pandas==1.1.5 
!pip install pandas==1.2.1

!pip install --upgrade google-cloud-bigquery
!pip install --upgrade db-dtypes
!pip install --upgrade pandas-gbq

!pip install pyradiomics

Found existing installation: pandas 1.2.1
Uninstalling pandas-1.2.1:
  Successfully uninstalled pandas-1.2.1
  Using cached pandas-1.2.1-cp37-cp37m-manylinux1_x86_64.whl (9.9 MB)


Import packages for both use cases. 

In [ ]:
import os
import sys
import shutil
import yaml
import time
import tqdm
import copy
import json 

from IPython.display import clear_output

# useful information
curr_dir = !pwd
curr_droid = !hostname
curr_pilot = !whoami

print(time.asctime(time.localtime()))

print("\nCurrent directory :", curr_dir[-1])
print("Hostname          :", curr_droid[-1])
print("Username          :", curr_pilot[-1])

print("Python version    :", sys.version.split('\n')[0])

Thu Aug 25 22:39:34 2022

Current directory : /content
Hostname          : f71feffedd67
Username          : root
Python version    : 3.7.12 (default, Jan 15 2022, 18:48:18) 


Import the other packages that nnUNet and BPR depend on. 

In [ ]:
%%capture

# Import packages etc that nnUnet depends on. 

!pip install SimpleITK
import SimpleITK as sitk
!pip install nnunet

import radiomics

# Import packages/repos that BPR depends on. 

from google.cloud import bigquery
import pandas as pd 
import pandas_gbq
import db_dtypes
from google.cloud import storage

start_time = time.time()

if os.path.isdir('/content/BodyPartRegression'):
  try:
    shutil.rmtree('/content/BodyPartRegression')
  except OSError as err:
    print("Error: %s : %s" % ("/content/BodyPartRegression", err.strerror)) 

!pip install torch==1.8.1 pytorch-lightning==1.2.10 torchtext==0.9.1 torchvision==0.9.1 torchaudio==0.8.1 dataclasses==0.6
!pip install opencv-python-headless==4.1.2.30 
!pip install bpreg
!git clone https://github.com/MIC-DKFZ/BodyPartRegression.git
!pip install pydicom
!pip install pyradiomics

import bpreg 
import seaborn as sb 
import glob
import matplotlib.pyplot as plt 
import pydicom
import subprocess
from collections import OrderedDict
from radiomics import featureextractor

from BodyPartRegression.docs.notebooks.utils import * 
from bpreg.scripts.bpreg_inference import bpreg_inference

In [ ]:
# Get revision number for bpreg 
!pip show bpreg > '/content/bpreg_output.txt'
f = open('/content/bpreg_output.txt')
d = {}
for line in f:
  (key, val) = line.split(': ')
  d[key] = val
bpr_revision_number = d['Version'].strip()

In [ ]:
print(bpr_revision_number)

1.1.0


Let's install and import the packages needed to create Structured Reports (SR). 

In [ ]:
# Packages for the structured report 

!pip uninstall highdicom
!git clone https://github.com/herrmannlab/highdicom.git
#!cd highdicom && python setup.py install
!cd highdicom && pip install .

import highdicom

from pathlib import Path

import highdicom as hd

from pydicom.uid import generate_uid
from pydicom.filereader import dcmread
from pydicom.sr.codedict import codes

from highdicom.sr.content import (
    FindingSite,
    ImageRegion,
    ImageRegion3D,
    SourceImageForRegion,
    SourceImageForMeasurement,
    SourceImageForMeasurementGroup
)
from highdicom.sr.enum import GraphicTypeValues3D
from highdicom.sr.enum import GraphicTypeValues
from highdicom.sr.sop import Comprehensive3DSR, ComprehensiveSR
from highdicom.sr.templates import (
    DeviceObserverIdentifyingAttributes,
    Measurement,
    MeasurementProperties,
    MeasurementReport,
    MeasurementsAndQualitativeEvaluations,
    ObservationContext,
    ObserverContext,
    PersonObserverIdentifyingAttributes,
    PlanarROIMeasurementsAndQualitativeEvaluations,
    RelationshipTypeValues,
    TrackingIdentifier,
    QualitativeEvaluation,
    ImageLibrary,
    ImageLibraryEntryDescriptors
)
from highdicom.sr.value_types import (
    CodedConcept,
    CodeContentItem,
)

import logging
logger = logging.getLogger("highdicom.sr.sop")
logger.setLevel(logging.INFO)

Found existing installation: highdicom 0.18.3
Uninstalling highdicom-0.18.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/highdicom-0.18.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/highdicom/*
Proceed (y/n)? y
  Successfully uninstalled highdicom-0.18.3
fatal: destination path 'highdicom' already exists and is not an empty directory.
Processing /content/highdicom
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for highdicom: filename=highdicom-0.18.3-py3-none-any.whl size=797271 sha256=cc90c9b2d638cff7eb97b95a413eac2a56f420a831643383fb5db5b6c86e0b2e
  Stored in directory: /tmp/pip-ephem-wheel-cache-n_1

Set the field for storage.

In [ ]:
bucket_base_uri =  "s3://%s/"%(bucket_name)


We will next install a number of packages needed for organizing and converting DICOM files:

1.    `dicomsort`, a package for sorting DICOM files into a directory tree using specific DICOM fields. 
2.   `plastimatch`, a package used to convert RTSTRUCT DICOM files to nrrd. 
3.   `dcmqi`, a package which converts SEG DICOM files to nrrd.
4.  `dcm2niix`, a package for converting DICOM files to nii 


In [ ]:
%%capture
start_time=time.time()

# dicomsort 
if os.path.isdir('/content/src/dicomsort'):
  try:
    shutil.rmtree('/content/src/dicomsort')
  except OSError as err:
    print("Error: %s : %s" % ("dicomsort", err.strerror)) 
# !git clone https://github.com/pieper/dicomsort.git 
!git clone https://github.com/pieper/dicomsort.git src/dicomsort

# plastimatch and pyplastimatch 
!sudo apt install plastimatch 
if os.path.isdir('/content/src/pyplastimatch'):
  try:
    shutil.rmtree('/content/src/pyplastimatch')
  except OSError as err:
    print("Error: %s : %s" % ("pyplastimatch", err.strerror)) 
!git clone https://github.com/AIM-Harvard/pyplastimatch src/pyplastimatch
import src.pyplastimatch.pyplastimatch.pyplastimatch as pypla

# dcmqi 
!wget https://github.com/QIICR/dcmqi/releases/download/v1.2.5/dcmqi-1.2.5-linux.tar.gz
!tar zxvf dcmqi-1.2.5-linux.tar.gz
!cp dcmqi-1.2.5-linux/bin/* /usr/local/bin/

# dcm2niix 
!sudo apt-get install dcm2niix 

end_time = time.time()
print ('time to install: ' + str(end_time-start_time))

We will install subversion so that we can clone code of a few repositories. 

In [ ]:
%%capture
!apt install subversion

Clone only the subfolders of `ImagingDataCommons/ai_medima_misc` we need to run this notebook.

In [ ]:
!svn checkout https://github.com/ImagingDataCommons/ai_medima_misc/trunk/nnunet/src
# !svn checkout https://github.com/ImagingDataCommons/ai_medima_misc/trunk/nnunet/data

Checked out revision 49.


Let's import the functions we need for preprocessing, processing and postprocessing of the data  

In [ ]:
import src.utils.gcs as gcs
import src.utils.preprocessing as preprocessing
import src.utils.processing as processing
import src.utils.postprocessing as postprocessing

Create directory tree for both

In [ ]:
# create the directory tree for both 

!mkdir -p data models output queries

!mkdir -p data/raw 
!mkdir -p data/raw/tmp 
!mkdir -p data/raw/nlst
!mkdir -p data/raw/nlst/dicom

# create the directory tree for nnUnet

!mkdir -p data/processed
!mkdir -p data/processed/nlst
!mkdir -p data/processed/nlst/nii
!mkdir -p data/processed/nlst/nnunet/nrrd
!mkdir -p data/processed/nlst/nnunet/dicomseg

!mkdir -p data/nnunet/model_input/
!mkdir -p data/nnunet/nnunet_output/

# create the directory free for bpr 

!mkdir -p data/bpr/model_input/
!mkdir -p data/bpr/model_input_tmp/
!mkdir -p data/bpr/bpr_output/
!mkdir -p data/bpr/bpr_output_tmp


Get the text files that store the queries needed to form the appropriate cohort.

In [ ]:
query_download_path = "/content/queries/query.txt"
!wget -O $query_download_path https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/common/queries/NLST_query.txt

--2022-08-25 22:44:37--  https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/common/queries/NLST_query.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6653 (6.5K) [text/plain]
Saving to: ‘/content/queries/query.txt’

/content/queries/qu 100%[===================>]   6.50K  --.-KB/s    in 0s      

2022-08-25 22:44:37 (58.4 MB/s) - ‘/content/queries/query.txt’ saved [6653/6653]



### nnUnet 

Copy the JSON metadata file (generated using [...]) from the repo.

In [ ]:
# bucket_data_base_uri = os.path.join(bucket_base_uri, "nnunet/data")
# dicomseg_json_uri = "s3://idc-medima-paper/nnunet/data/dicomseg_metadata.json"
# dicomseg_json_path = "/content/data/dicomseg_metadata.json"

# !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $dicomseg_json_uri $dicomseg_json_path

dicomseg_json_path = "/content/data/dicomseg_metadata.json"
!wget -N -P '/content/data' https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/nnunet/data/dicomseg_metadata.json

--2022-08-25 22:44:37--  https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/nnunet/data/dicomseg_metadata.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2706 (2.6K) [text/plain]
Saving to: ‘/content/data/dicomseg_metadata.json’

dicomseg_metadata.j 100%[===================>]   2.64K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2022-08-25 22:44:37 (63.9 MB/s) - ‘/content/data/dicomseg_metadata.json’ saved [2706/2706]



Download the segmentation models:

In [ ]:
model_download_path = "/content/models/Task055_SegTHOR.zip"
!$s5cmd_path --endpoint-url https://storage.googleapis.com cp s3://idc-medima-paper/nnunet/model/Task055_SegTHOR.zip $model_download_path

# model_download_path = "/content/models/Task055_SegTHOR.zip"
# !wget -O $model_download_path https://zenodo.org/record/4003545/files/Task055_SegTHOR.zip?download=1 # approx 6 minutes, but sometimes 10+ using colab vm


cp s3://idc-medima-paper/nnunet/model/Task055_SegTHOR.zip /content/models/Task055_SegTHOR.zip


Initialize a few environment variables needed for nnUNet.

In [ ]:
os.environ["RESULTS_FOLDER"] = "/content/data/nnunet/nnunet_output/"
os.environ["WEIGHTS_FOLDER"] = "/content/data/nnunet/nnunet_output/nnUNet"

Install the pretrained nnUNet model. 

In [ ]:
# %%capture
!nnUNet_install_pretrained_model_from_zip $model_download_path



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

nnUNet_raw_data_base is not defined and nnU-Net can only be used on data for which preprocessed files are already present on your system. nnU-Net cannot be used for experiment planning and preprocessing like this. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up properly.
nnUNet_preprocessed is not defined and nnU-Net can not be used for preprocessing or training. If this is not intended, please read documentation/setting_up_paths.md for information on how to set this up.


Get the csv that will contain the mapping for the nnUNet features -- this is needed for the creation of the structured report. 

In [ ]:
!wget -N https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/nnunet/data/nnunet_segments_code_mapping.csv
nnunet_segments_code_mapping_df = pd.read_csv("nnunet_segments_code_mapping.csv")

!wget -N https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/nnunet/data/nnunet_shape_features_code_mapping.csv
nnunet_shape_features_code_mapping_df = pd.read_csv("nnunet_shape_features_code_mapping.csv")

--2022-08-25 22:45:51--  https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/nnunet/data/nnunet_segments_code_mapping.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 355 [text/plain]
Saving to: ‘nnunet_segments_code_mapping.csv’

nnunet_segments_cod 100%[===================>]     355  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2022-08-25 22:45:52 (12.0 MB/s) - ‘nnunet_segments_code_mapping.csv’ saved [355/355]

--2022-08-25 22:45:52--  https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/nnunet/data/nnunet_shape_features_code_mapping.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.g

### BPR 

Download the BPR model from zenodo. 

In [ ]:
bpr_model_url = "https://zenodo.org/record/5113483/files/public_bpr_model.zip"
model_download_path = "/content/models/bpr_model.zip"

!$s5cmd_path --endpoint-url https://storage.googleapis.com cp s3://idc-medima-paper/bpr/model/bpr_model.zip $model_download_path 

cp s3://idc-medima-paper/bpr/model/bpr_model.zip /content/models/bpr_model.zip


Unzip the model 

In [ ]:
model_extract_path = "/content/models/bpr_model"
!unzip $model_download_path -d $model_extract_path

Archive:  /content/models/bpr_model.zip
replace /content/models/bpr_model/public_bpr_model/reference.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/models/bpr_model/public_bpr_model/reference.xlsx  
  inflating: /content/models/bpr_model/public_bpr_model/inference-settings.json  
  inflating: /content/models/bpr_model/public_bpr_model/model.pt  
  inflating: /content/models/bpr_model/public_bpr_model/config.json  


Get the csv that will contain the mapping for the BPR regions - this is needed for the creation of the structured report. 

In [ ]:
!wget -N https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/bpr/data/bpr_regions_code_mapping.csv
bpr_regions_df = pd.read_csv("bpr_regions_code_mapping.csv")

--2022-08-25 22:50:14--  https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/bpr/data/bpr_regions_code_mapping.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 444 [text/plain]
Saving to: ‘bpr_regions_code_mapping.csv’

bpr_regions_code_ma 100%[===================>]     444  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2022-08-25 22:50:14 (33.5 MB/s) - ‘bpr_regions_code_mapping.csv’ saved [444/444]



Get the csv that will contain the mapping for the BPR landmarks -- this is needed for the creation of the structured report. 

In [ ]:
!wget -N https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/bpr/data/bpr_landmarks_code_mapping.csv
landmarks_df = pd.read_csv("bpr_landmarks_code_mapping.csv")

--2022-08-25 22:50:14--  https://raw.githubusercontent.com/ImagingDataCommons/ai_medima_misc/main/bpr/data/bpr_landmarks_code_mapping.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1749 (1.7K) [text/plain]
Saving to: ‘bpr_landmarks_code_mapping.csv’

bpr_landmarks_code_ 100%[===================>]   1.71K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2022-08-25 22:50:14 (48.2 MB/s) - ‘bpr_landmarks_code_mapping.csv’ saved [1749/1749]



---

---

# **Function Definition**

## **Data Download and Preparation**

The following function handles the download of a single patient data from the IDC buckets using `gsutil cp`. Furthermore, to organise the data in a more human-understandable and, above all, standardized fashion, the function makes use of [DICOMSort](https://github.com/pieper/dicomsort).

DICOMSort is an open source tool for custom sorting and renaming of dicom files based on their specific DICOM tags. In our case, we will exploit DICOMSort to organise the DICOM data by `PatientID` and `Modality` - so that the final directory will look like the following:

```
raw/nsclc-radiomics/dicom/$PatientID
 └─── CT
       ├─── $SOPInstanceUID_slice0.dcm
       ├─── $SOPInstanceUID_slice1.dcm
       ├───  ...
       │
      RTSTRUCT 
       ├─── $SOPInstanceUID_RTSTRUCT.dcm
      SEG
       └─── $SOPInstanceUID_RTSEG.dcm

```

In [ ]:
def download_series_data_s5cmd(raw_base_path, 
                               sorted_base_path,
                               series_df, 
                               remove_raw = True):

  """
  Download raw DICOM data and run dicomsort to standardise the input format.
  Uses s5cmd instead of gsutil which allows for a faster download. 

  Arguments:
    raw_base_path    : required - path to the folder where the raw data will be stored.
    sorted_base_path : required - path to the folder where the sorted data will be stored.
    series_df        : required - Pandas dataframe (returned from BQ) storing all the
                                  series information required to pull data from the IDC buckets.
    remove_raw       : optional - whether to remove or not the raw non-sorted data
                                  (after sorting with dicomsort). Defaults to True.
  
  Outputs:
    This function [...]
  """

  # Get and create the download path 
  series_id = series_df["SeriesInstanceUID"].values[0]
  download_path = os.path.join(raw_base_path, series_id)
  if not os.path.exists(download_path):
    os.mkdir(download_path)

  # Create the text file to hold gsc_url 
  gcsurl_temp = "cp " + series_df["gcs_url"].str.replace("gs://","s3://") + " " + download_path 
  gs_file_path = "gcs_paths.txt"
  gcsurl_temp.to_csv(gs_file_path, header = False, index = False)

  # Get the path for s5cmd
  if os.path.exists('/content/s5cmd'):
    s5cmd_path = '/content/s5cmd'
  else:
    s5cmd_path = 's5cmd'

  # Download using s5cmd 
  start_time = time.time()
  # download_cmd = ["/content/s5cmd","--endpoint-url", "https://storage.googleapis.com", "run", gs_file_path]
  # download_cmd = ["s5cmd","--endpoint-url", "https://storage.googleapis.com", "run", gs_file_path]
  download_cmd = [s5cmd_path,"--endpoint-url", "https://storage.googleapis.com", "run", gs_file_path]
  proc = subprocess.Popen(download_cmd)
  proc.wait()
  elapsed = time.time() - start_time 
  print ("Done download in %g seconds."%elapsed)

  # Sort files 
  start_time = time.time()
  print("\nSorting DICOM files..." )
  # !python src/dicomsort/dicomsort.py -u $download_path $sorted_base_path/%PatientID/%Modality/%SOPInstanceUID.dcm
  !python src/dicomsort/dicomsort.py -u $download_path $sorted_base_path/%SeriesInstanceUID/%Modality/%SOPInstanceUID.dcm
  elapsed = time.time() - start_time
  print("Done sorting in %g seconds."%elapsed)

  print("Sorted DICOM data saved at: %s"%(os.path.join(sorted_base_path, series_id)))

  # get rid of the temporary folder, storing the unsorted DICOM data 
  if remove_raw:
    print("Removing un-sorted data at %s..."%(download_path))
    !rm -r $download_path
    print("... Done.")

---

## **Data Preprocessing**

Brief description here.



In [ ]:
def dcm2niix_dicom_ct_to_nifti(sorted_base_path,
                               processed_nifti_path,
                               pat_id):
  
  """
    This function converts the sorted DICOM patient data to a NIfTI file 
    (CT volume) using dcm2niix. 

    Arguments:
      sorted_base_path     : required - path to the folder where the sorted data should be stored.
      processed_nifti_path : required - path to the folder where the preprocessed NIfTI data are stored
      pat_id               : required - patient ID (used for naming purposes).
    
    Outputs:
      writes the NIfTI file os.path.join(processed_nifti_path, pat_id, pat_id + 'CT.nii.gz') out 
      
    """

  success = 0

  # given that everything is standardised already, compute the paths
  path_to_dicom_ct_folder = os.path.join(sorted_base_path, pat_id, "CT")
  
  # sanity check
  assert(os.path.exists(path_to_dicom_ct_folder))
  
  pat_dir_nifti_path = os.path.join(processed_nifti_path, pat_id)
  if not os.path.exists(pat_dir_nifti_path):
    os.mkdir(pat_dir_nifti_path)

  # output nii CT
  ct_nifti_path = os.path.join(pat_dir_nifti_path, pat_id + "_CT.nii.gz")

  # DICOM CT to nii conversion 
  if not os.path.exists(ct_nifti_path):
    cmd = 'dcm2niix -z y -m y -o %s  %s ' % (pat_dir_nifti_path, path_to_dicom_ct_folder)
    print(cmd)
    ret = os.system(cmd)
    print (ret)
    num_files = len([f for f in os.listdir(pat_dir_nifti_path) if f.endswith('.nii.gz')])
    print('num_files: ' + str(num_files))

    # If only one file created, keep it. 
    if num_files == 1: 
      success = 0 
      # rename the file 
      src = os.path.join(pat_dir_nifti_path, [f for f in os.listdir(pat_dir_nifti_path) if f.endswith('.nii.gz')][0]) 
      os.rename(src, ct_nifti_path)  
    # For each file, if multiple are created, find the one that has the same number of slices as dicom files 
    else: 
      file_names = [os.path.join(pat_dir_nifti_path,f) for f in os.listdir(pat_dir_nifti_path) if f.endswith('.nii.gz')]
      num_slices = [nib.load(f).get_fdata().shape[2] for f in file_names] 
      num_dicom_files = len(os.listdir(path_to_dicom_ct_folder))
      success = -1 
      if num_dicom_files in num_slices:
        index = num_slices.index(num_dicom_files) 
        src = file_names[index]
        # rename the file
        os.rename(src, ct_nifti_path)
        success = 0  

    # if num_files != 1: 
    #   success = -1 
    # else: 
    #   success = 0
    #   # rename the file 
    #   src = os.path.join(pat_dir_nifti_path, [f for f in os.listdir(pat_dir_nifti_path) if f.endswith('.nii.gz')][0]) 
    #   os.rename(src, ct_nifti_path)  

  return success 


In [ ]:
def prep_input_data_bpr(processed_nifti_path, model_input_folder, pat_id):
  
  """
  Copies the nifti file to the appropriate model_input_folder

  Arguments:
    processed_nifti_path : required - path to the folder where the input nifti are stored 
    model_input_folder   : required - path to the folder where the nifti files for BPR will be stored
    pat_id               : required - patient ID (used for naming purposes).
  
  Outputs:
    This function [...]
  """

  # FIXME: ok for a notebook; for scripting, change this to `shutil`

  pat_dir_nifti_path = os.path.join(processed_nifti_path, pat_id)
  ct_nifti_path = os.path.join(pat_dir_nifti_path, pat_id + "_CT.nii.gz")
  
  # copy_to_path = os.path.join(model_input_folder, pat_id + "_0000.nii.gz")
  # copy_to_path = os.path.join(model_input_folder, pat_id + "_CT.nii.gz")
  copy_to_path = os.path.join(model_input_folder, pat_id + ".nii.gz")
    
  # copy NIfTI to the right dir for bpr processing
  if not os.path.exists(copy_to_path):
    print("Copying %s\nto %s..."%(ct_nifti_path, copy_to_path))
    !cp $ct_nifti_path $copy_to_path
    print("... Done.")

---

## **Data Processing**

Brief description here.

In [ ]:
def process_patient_bpr(model_input_folder, model_output_folder, model):
  
  """
  Predict the body part region scores from the model_input_folder. 

  Arguments:
    model_input_folder  : required - path to the folder where the data to be inferred should be stored.
    model_output_folder : required - path to the folder where the output json files will be stored.

  Outputs:
    This function produces a json file per input nifti that stores the body part scores. 
  """

  start_time = time.time()
  # bpreg_inference(model_input_folder, model_output_folder)
  bpreg_inference(model_input_folder, model_output_folder, model)
  end_time = time.time()
  elapsed = end_time-start_time 
  print("Done in %g seconds."%elapsed)


  return 

---

## **Data Postprocessing**

Description here.

---

---

## **General Utilities**

In [ ]:
def file_exists_in_bucket_s3(project_name, bucket_name, file_gs_uri):
  
  """
  Check whether a file exists in the specified Google Cloud Storage Bucket.

  Arguments:
    project_name : required - name of the GCP project.
    bucket_name  : required - name of the bucket (without gs://)
    file_gs_uri  : required - file GS URI
  
  Returns:
    file_exists : boolean variable, True if the file exists in the specified,
                  bucket, at the specified location; False if it doesn't.

  Outputs:
    This function [...]
  """

  storage_client = storage.Client(project = project_name)
  bucket = storage_client.get_bucket(bucket_name)
  
  # bucket_gs_url = "gs://%s/"%(bucket_name)
  bucket_gs_url = "s3://%s/"%(bucket_name)
  path_to_file_relative = file_gs_uri.split(bucket_gs_url)[-1]

  print("Searching `%s` for: \n%s\n"%(bucket_gs_url, path_to_file_relative))

  file_exists = bucket.blob(path_to_file_relative).exists(storage_client)

  return file_exists

---


In [ ]:
def listdir_bucket_s3(project_name, bucket_name, dir_gs_uri):
  
  """
  Export DICOM SEG object from segmentation masks stored in NRRD files.

  Arguments:
    project_name : required - name of the GCP project.
    bucket_name  : required - name of the bucket (without gs://)
    file_gs_uri  : required - directory GS URI
  
  Returns:
    file_list : list of files in the specified GCS bucket.

  Outputs:
    This function [...]
  """

  storage_client = storage.Client(project = project_name)
  bucket = storage_client.get_bucket(bucket_name)
  
  # bucket_gs_url = "gs://%s/"%(bucket_name)
  bucket_gs_url = "s3://%s/"%(bucket_name)
  path_to_dir_relative = dir_gs_uri.split(bucket_gs_url)[-1]


  print("Getting the list of files at `%s`..."%(dir_gs_uri))

  file_list = list()

  for blob in storage_client.list_blobs(bucket_name,  prefix = path_to_dir_relative):
    fn = os.path.basename(blob.name)
    file_list.append(fn)

  return file_list

In [ ]:
def create_dataset(project_name, dataset_id):

  """
  Create a dataset that will store the cohort_df table 

  Arguments:
    project_name : required - name of the GCP project.
    dataset_id   : required - name of the dataset to create
  
  Returns:
    dataset : returns the dataset created 
  """

  # Construct a BigQuery client object.
  client = bigquery.Client(project=project_name)

  # Construct a full Dataset object to send to the API.
  dataset_id_full = ".".join([project_name, dataset_id])
  dataset = bigquery.Dataset(dataset_id_full)

  # TODO(developer): Specify the geographic location where the dataset should reside.
  dataset.location = "US"

  # Send the dataset to the API for creation, with an explicit timeout.
  # Raises google.api_core.exceptions.Conflict if the Dataset already
  # exists within the project.
  # dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
  dataset = client.create_dataset(dataset)  # Make an API request.

  print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

  return dataset 


In [ ]:
def dataset_exists_in_project(project_id, dataset_id):
  """Check if a dataset exists in a project"""

  from google.cloud import bigquery
  from google.cloud.exceptions import NotFound

  client = bigquery.Client()
  dataset_id_full = '.'.join([project_id, dataset_id])

  try:
      client.get_dataset(dataset_id_full)  
      return True 
  except NotFound:
      return False 

In [ ]:
def create_dataset(project_name, dataset_id):

  """
  Create a dataset that will store the cohort_df table 

  Arguments:
    project_name : required - name of the GCP project.
    dataset_id   : required - name of the dataset to create
  
  Returns:
    dataset : returns the dataset created 
  """

  # Construct a BigQuery client object.
  client = bigquery.Client(project=project_name)

  # Construct a full Dataset object to send to the API.
  dataset_id_full = ".".join([project_name, dataset_id])
  dataset = bigquery.Dataset(dataset_id_full)

  # TODO(developer): Specify the geographic location where the dataset should reside.
  dataset.location = "US"

  # Send the dataset to the API for creation, with an explicit timeout.
  # Raises google.api_core.exceptions.Conflict if the Dataset already
  # exists within the project.
  # dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
  dataset = client.create_dataset(dataset)  # Make an API request.

  print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

  return dataset 


In [ ]:
def get_dataframe_from_table(project_name, dataset_id, table_id_name):
  
  """
  Gets the pandas dataframe from the saved big query table 

  Arguments:
    project_name : required - name of the GCP project.
    dataset_id   : required - name of the dataset already created 
    table_id     : required - name of the table to create 
  
  Returns: 
    cohort_df    : the cohort as a pandas dataframe
  """

  table_id = "%s.%s.%s"%(project_name, dataset_id, table_id_name)

  # the query we are going to execute to gather data about the selected cohort
  query_str = "SELECT * FROM `%s`"%(table_id)

  # init the BQ client
  client = bigquery.Client(project = project_name)

  # run the query
  query_job = client.query(query_str)

  # convert the results to a Pandas dataframe
  # cohort_df = query_job.to_dataframe()
  cohort_df = query_job.to_dataframe(create_bqstorage_client=True)


  return cohort_df 


---

In [ ]:
def modify_dicomseg_json_file(dicomseg_json_path, dicomseg_json_path_modified, SegmentAlgorithmName):

  """
  This function writes out a new metadata json file for the DICOM Segmentation object. 
  It sets the SegmentAlgorithmName to the one provided as input. 

  Arguments:
    dicomseg_json_path          : path of the original dicomseg json file 
    dicomseg_json_path_modified : the new json file to write to disk 
    SegmentAlgorithmName        : the field to replace
    
  Returns:
    The json file is written out to dicomseg_json_path_modified 

  """
  f = open(dicomseg_json_path)
  meta_json = json.load(f)

  meta_json_modified = copy.deepcopy(meta_json)
  num_regions = len(meta_json_modified['segmentAttributes'])
  for n in range(0,num_regions): 
    meta_json_modified['segmentAttributes'][n][0]['SegmentAlgorithmName'] = SegmentAlgorithmName

  with open(dicomseg_json_path_modified, 'w') as f: 
    json.dump(meta_json_modified, f)

  return 

  # dicomseg_json_uri = "s3://idc-medima-paper/nnunet/data/dicomseg_metadata.json"
  # dicomseg_json_path = "/content/data/dicomseg_metadata.json"



In [ ]:
def append_row_to_bq_table_with_query(project_name, dataset_name, table_name, value_SeriesInstanceUID, row_to_insert):

  bpr_table_id_fullname = '.'.join([project_name, dataset_name, table_name])

  query = f"""
    SELECT 
      COUNT(SeriesInstanceUID) AS num_instances
    FROM 
      {bpr_table_id_fullname}
    WHERE 
      SeriesInstanceUID IN UNNEST (@pat_id_change);
  """
  job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ArrayQueryParameter("pat_id_change", "STRING", [value_SeriesInstanceUID])]) 
  client = bigquery.Client(project=project_name)
  result = client.query(query, job_config=job_config)
  df_result = result.to_dataframe()
  count = df_result['num_instances'][0]
  print(count)
  
  if (count>=1):
    print("Cannot insert row because seriesInstanceUID exists in table")
  else:
    print("Inserting row into table")
    client = bigquery.Client(project=project_name)
    dataset = client.dataset(dataset_name)
    table_ref = dataset.table(table_name)
    client.insert_rows_json(table = table_ref,
                            json_rows = row_to_insert,
                            skip_invalid_rows = False,
                            ignore_unknown_values = False)

  return 

## GCP healthcare functions

In [ ]:
def create_gcp_healthcare_dataset(project_name, location_id, dataset_id):
  """Creates a GCP healthcare dataset using gcloud commands if it doesn't exist

  Inputs: 
    project_name : the project id 
    location_id  : the location of the project, e.g. us-central1
    dataset_id   : the name of the dataset to create 

  Outputs:
    creates the dataset if it doesn't exist 

  """
  
  # First let's list the datasets that we already have for our particular project_id and location_id
  datasets = !gcloud healthcare datasets list --project $project_name --location $location_id --format="value(ID)" 
  print ('datasets that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ': ' + str(datasets))
  
  # If the dataset doesn't exist, create it 
  if not (dataset_id in datasets):
    try:
      !gcloud healthcare datasets create --project $project_name $dataset_id --location $location_id
      print ("Created dataset " + str(dataset_id))
    except OSError as err:
      print("Error: %s : %s" % ("Unable to create dataset", err.strerror)) 

  # As a check, we'll list the datasets again.
  datasets = !gcloud healthcare datasets list --project $project_name --location $location_id --format="value(ID)" 
  print ('datasets that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ': ' + str(datasets))

  return

In [ ]:
def create_gcp_healthcare_dicom_datastore(project_name, location_id, dataset_id, datastore_id):
  """Creates a GCP healthcare DICOM datastore using gcloud commands if it doesn't exist

  Inputs: 
    project_name : the project id 
    location_id  : the location of the project, e.g. us-central1
    dataset_id   : the dataset id
    datastore_id : the DICOM datastore to create 

  Outputs:
    creates the DICOM datastore if it doesn't exist

  """

  # First list the datastores that exist in the dataset
  datastores = !gcloud healthcare dicom-stores list --project $project_name --location $location_id --dataset $dataset_id --format="value(ID)"
  print ('datastores that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ', dataset ' + str(dataset_id) + ': ' + str(datastores))

  # If the dicom_store_id doesn't exist, create it
  if not (datastore_id in datastores):
    try:
      !gcloud healthcare dicom-stores create $datastore_id --project $project_name --location $location_id --dataset $dataset_id --format="value(ID)" 
      print ("Created DICOM datastore " + str(datastore_id))
    except OSError as err:
      print("Error: %s : %s" % ("Unable to create datastore", err.strerror)) 

  # As a check, we'll list the datastores again.
  datastores = !gcloud healthcare dicom-stores list --project $project_name --location $location_id --dataset $dataset_id --format="value(ID)"
  print ('datastores that exist for project_id ' + str(project_name) + ', location ' + str(location_id) + ', dataset ' + str(dataset_id) + ': ' + str(datastores))

  return

In [ ]:
def create_ohif_viewer_2server_url(my_ohif_app, project_name, location_id, dataset_id, datastore_id, study_id): 
  """Forms the url for the 2 server OHIF viewer app 

  Inputs: 
    my_ohif_app  : name of the app 
    project_name : the project id 
    location_id  : the location of the project, e.g. us-central1
    dataset_id   : the dataset id
    datastore_id : the DICOM datastore to create 
    study_id     : the study id where the raw CT data is located 

  Outputs:
    creates the OHIF viewer url, e.g 
    https://idc2serversdeploy-3c769.web.app/viewer/1.2.840.113654.2.55.171889818252959598680709500375734141432!secondGoogleServer=/projects/idc-external-018/locations/us-central1/datasets/dataset_nlst/dicomStores/2d-tta-seg-only

  """

  ohif_viewer_url = os.path.join("https://", 
                                 my_ohif_app + '.web.app', 
                                 'viewer', 
                                 study_id + '!secondGoogleServer=',
                                 'projects', project_name, 
                                 'locations', location_id, 
                                 'datasets', dataset_id, 
                                 'dicomStores', datastore_id)

  return ohif_viewer_url 
  

In [ ]:
def create_ohif_viewer_url(my_ohif_app, project_name, location_id, dataset_id, datastore_id, study_id): 
  """Forms the url for the OHIF viewer app 

  Inputs: 
    my_ohif_app  : name of the app 
    project_name : the project id 
    location_id  : the location of the project, e.g. us-central1
    dataset_id   : the dataset id
    datastore_id : the DICOM datastore to create 
    study_id     : the study id where the raw CT data is located 

  Outputs:
    creates the OHIF viewer url, e.g 
    https://idc-tester-dk.web.app/projects/idc-external-018/locations/us-central1/datasets/siim_cmimi22_dataset/dicomStores/siim_cmimi22_datastore/study/1.3.6.1.4.1.32722.99.99.62087908186665265759322018723889952421
  

  """

  ohif_viewer_url = os.path.join("https://", 
                                 my_ohif_app + '.web.app', 
                                 'projects', project_name, 
                                 'locations', location_id, 
                                 'datasets', dataset_id, 
                                 'dicomStores', datastore_id, 
                                 'study', study_id)

  return ohif_viewer_url 
  

## Cleanup

In [ ]:
def clean_files_in_nnunet_path(series_id, processed_base_path, model_output_folder_nnunet, processed_nrrd_path_nnunet):

  """Cleans up the nnUnet files for the next run

  Inputs: 
    series_id                  : series_id being processed 
    processed_base_path        : base path to hold processed results -- e.g. "/content/data/processed/nlst/"
    model_output_folder_nnunet : output results for nnuNet -- e.g. "/content/data/nnunet/nnunet_output/"
    processed_nrrd_path_nnunet : the nrrd results of nnUNet -- e.g. os.path.join(processed_base_path, "nnunet", "nrrd")

  Outputs:
    cleans the files in the respective directories 

  """

  # delete the files in the pat_dir_dicomseg_path
  processed_dicomseg_path = os.path.join(processed_base_path, "nnunet", "dicomseg")
  pat_dir_dicomseg_path = os.path.join(processed_dicomseg_path, series_id)
  # print ('removing files from: ' + pat_dir_dicomseg_path)
  !rm -r $pat_dir_dicomseg_path/*
  !rmdir $pat_dir_dicomseg_path

  # remove files from the nnunet output prediction 
  !rm $model_output_folder_nnunet/*.nii.gz
  !rm $model_output_folder_nnunet/*.npz
  !rm $model_output_folder_nnunet/*.pkl
  !rm $model_output_folder_nnunet/*.json

  # delete the files in the processed_nrrd_path_nnunet
  processed_nrrd_path_nnunet_segments = os.path.join(processed_nrrd_path_nnunet,series_id,'pred_softmax')
  !rm $processed_nrrd_path_nnunet_segments/*.nrrd
  !rm $processed_nrrd_path_nnunet/$series_id/*.log
  !rm $processed_nrrd_path_nnunet/$series_id/*_pred_segthor*.nrrd

  # delete the series folder in /content/data/processed/nlst/nnunet/nrrd/series_id
  nrrd_dir = os.path.join(processed_base_path,'nnunet','nrrd',series_id)
  !rmdir $nrrd_dir

  return 

In [ ]:
def clean_files_in_bpr_path(series_id, 
                            model_input_folder_bpr, 
                            model_input_folder_tmp_bpr, 
                            model_output_folder_tmp_bpr, 
                            model_output_folder_sr_regions_bpr, 
                            model_output_folder_sr_landmarks_bpr,
                            input_nifti_path, 
                            sorted_base_path, 
                            processed_nifti_path):
  
  """Cleans up the BPR files for the next run

  Inputs: 
    series_id                            : series_id being processed 
    model_input_folder_bpr               : input used for BPR 
    model_input_folder_tmp_bpr           : temp folder for input bpr results 
    model_output_folder_tmp_bpr          : temp folder for output bpr results 
    model_output_folder_sr_regions_bpr   : folder for SR dcm files (regions)
    model_output_folder_sr_landmarks_bpr : folder for SR dcm files (landmarks)
    input_nifti_path                     : the nifti file BPR was run on 
    sorted_base_path                     : where the raw DICOM files are stored 
    processed_nifti_path                 : where the processed nifti files are stored 

  Outputs:
    cleans the files in the respective directories 

  """
  
  # remove the file from the model_input_tmp folder
  model_input_folder_tmp_series = os.path.join(model_input_folder_tmp_bpr,series_id+'.nii.gz')
  !rm $model_input_folder_tmp_series

  # remove the file from the model_output_tmp folder 
  model_output_folder_tmp_series = os.path.join(model_output_folder_tmp_bpr,series_id+'.json')
  !rm $model_output_folder_tmp_series

  # remove the NIfTI file the prediction was computed from
  !rm $input_nifti_path

  # Remove the raw patient data 
  sorted_base_path_remove = os.path.join(sorted_base_path,series_id)
  !rm -r $sorted_base_path_remove 

  # Delete json from bpr_output folder 
  model_output_folder_series = os.path.join(model_output_folder_bpr,series_id+'.json')
  !rm $model_output_folder_series

  # Delete the files in /content/data/processed/bpr/nii
  # !rm -r /content/data/processed/bpr/nii

  # remove from model input 
  # !rm /content/data/nnunet/model_input/*

  # remove from processed path
  !rm -r /content/data/processed/nlst/nii/*

  # remove SR regions folder contents
  # !rm -r $model_output_folder_sr_regions_bpr
  !rm -r $model_output_folder_sr_regions_bpr/*

  # remove SR landmarks folder contents
  # !rm -r $model_output_folder_sr_landmarks_bpr
  !rm -r $model_output_folder_sr_landmarks_bpr/*


  return 


## BPR regions SR creation

In [ ]:
# takes as input a json file and slice_index, returns the regions assigned to the slice 
def convert_slice_to_region(bpr_data, slice_index):

  """ 
  Given the slice_index, this returns a list of corresponding regions that match

  Inputs: 
    bpr_data    : a dictionary, where for each of the six regions, a list of 
                  slice indices are provided 
    slice_index : slice number you want to obtain the list of classified regions 
                  for
  Returns
    regions     : list of regions for that slice_index

  """

  # find where the slice_index appears across all regions
  # get the names of the regions

  num_regions = len(bpr_data)
  region_names = list(bpr_data.keys())
  regions = [] 

  for n in range(0,num_regions):
    vals = bpr_data[region_names[n]]
    if slice_index in vals:
      regions.append(region_names[n])

  return regions 


In [ ]:
# # Convert a list of regions (single slice) to a qualitative_evaluations 
# def convert_regions_list_to_qualitative_evaluations(regions):

#   """ 
#   Forms the qualitative_evalautions array needed for the creation of the 
#   Structured Report. This is for a particular set of regions. 

#   Inputs: 
#     regions                    : list of regions that a particular slice
#                                  includes 
#   Returns
#     qualitative_evaluations    : array of the qualitative_evaluations. 

#   """

#   qualitative_evaluations = [] 
#   num_regions = len(regions)

#   for region in regions: 
    
#     if (region=="legs"):
#       qualitative_evaluations.append(QualitativeEvaluation(CodedConcept(
#         value="123014",
#         meaning="Target Region",
#         scheme_designator="DCM"
#       ), 
#       CodedConcept(
#           value="30021000",
#           meaning="Legs",
#           scheme_designator="SCT"
#       )
#       #, RelationshipTypeValues.CONTAINS
#       ))

#     elif (region=="pelvis"):
#       qualitative_evaluations.append(QualitativeEvaluation(CodedConcept(
#         value="123014",
#         meaning="Target Region",
#         scheme_designator="DCM"
#       ), 
#       CodedConcept(
#           value="12921003",
#           meaning="Pelvis",
#           scheme_designator="SCT"
#       )
#       #, RelationshipTypeValues.CONTAINS
#       ))

#     elif (region=="abdomen"):
#       qualitative_evaluations.append(QualitativeEvaluation(CodedConcept(
#         value="123014",
#         meaning="Target Region",
#         scheme_designator="DCM"
#       ), 
#       CodedConcept(
#           value="113345001",
#           meaning="Abdomen",
#           scheme_designator="SCT"
#       )
#       #, RelationshipTypeValues.CONTAINS
#       ))

#     elif (region=="chest"):
#       qualitative_evaluations.append(QualitativeEvaluation(CodedConcept(
#         value="123014",
#         meaning="Target Region",
#         scheme_designator="DCM"
#       ), 
#       CodedConcept(
#           value="51185008",
#           meaning="Chest",
#           scheme_designator="SCT"
#       )
#       #, RelationshipTypeValues.CONTAINS
#       ))

#     elif (region=="shoulder-neck"):
#       qualitative_evaluations.append(QualitativeEvaluation(CodedConcept(
#         value="123014",
#         meaning="Target Region",
#         scheme_designator="DCM"
#       ), 
#       CodedConcept(
#           value="45048000",
#           meaning="Neck",
#           scheme_designator="SCT"
#       )
#       #, RelationshipTypeValues.CONTAINS
#       ))

#     elif (region=="head"):
#       qualitative_evaluations.append(QualitativeEvaluation(CodedConcept(
#         value="123014",
#         meaning="Target Region",
#         scheme_designator="DCM"
#         ), 
#         CodedConcept(
#             value="69536005",
#             meaning="Head",
#             scheme_designator="SCT"
#         )
#         #, RelationshipTypeValues.CONTAINS
#         ))



#   return qualitative_evaluations 

In [ ]:
def create_structured_report_for_body_part_regression_regions(files, 
                                                              json_file, 
                                                              output_SR_file, 
                                                              bpr_revision_number,
                                                              bpr_regions_df):

  """Takes as input a set of DICOM files and the corresponding body part regression json file, 
     and writes a structured report (SR) to disk
     
  Inputs: 
    files               : list of CT dicom files 
    json_file           : the json file created from the BodyPartRegression prediction
    output_SR_file      : output filename for the structured report 
    bpr_revision_number : specific revision number of the bpr repo 
    bpr_regions_df      : holds the metadata needed for the bpr target regions 

  Outputs:
    writes the SR out to the output_SR_file.    
     
  """


  # ------ order the CT files according to the ImagePositionPatient and ImageOrientation ----# 

  num_files = len(files)
  # print ("num_files: " + str(num_files))

  pos_all = []  
  sop_all = [] 

  for n in range(0,num_files):
    # read dcm file 
    filename = files[n]
    ds = dcmread(filename)
    # print(ds)

    # get ImageOrientation (0020, 0037)
    # print(ds['0x0020','0x0037'].value)
    ImageOrientation = ds['0x0020','0x0037'].value

    # get ImagePositionPatient (0020, 0032) 
    ImagePositionPatient = ds['0x0020','0x0032'].value

    # calculate z value
    x_vector = ImageOrientation[0:3]
    y_vector = ImageOrientation[3:]
    z_vector = np.cross(x_vector,y_vector)

    # multiple z_vector by ImagePositionPatient
    pos = np.dot(z_vector,ImagePositionPatient)
    pos_all.append(pos)

    # get the SOPInstanceUID 
    sop = ds['0x0008', '0x0018'].value
    sop_all.append(sop)


  #----- order the SOPInstanceUID/files by z value ----# 

  sorted_ind = np.argsort(pos_all)
  pos_all_sorted = np.array(pos_all)[sorted_ind.astype(int)]
  sop_all_sorted = np.array(sop_all)[sorted_ind.astype(int)]
  files_sorted = np.array(files)[sorted_ind.astype(int)]

  #---- Open the json file and parse the list of regions per slice -----# 

  f = open(json_file)
  json_data = json.load(f)
  bpr_data = json_data['body part examined']

  # return a list where each entry is per slice and is a array of possible regions 
  bpr_slice_scores = json_data['cleaned slice scores']
  num_slices = len(bpr_slice_scores)

  num_regions = len(bpr_data)
  regions = [] 

  # print('num_slices: ' + str(num_slices))

  for slice_index in range(0,num_slices):
    region = convert_slice_to_region(bpr_data, slice_index)
    regions.append(region)

  # ----- Create the structured report ----- # 

  # Create the report content

  procedure_code = CodedConcept(value="363679005", scheme_designator="SCT", 
                                meaning="Imaging procedure")

  # Describe the context of reported observations: the person that reported
  # the observations and the device that was used to make the observations
  observer_person_context = ObserverContext(
      observer_type=codes.DCM.Person,
      observer_identifying_attributes=PersonObserverIdentifyingAttributes(
          name='Anonymous^Reader'
      )
  )
  # observer_device_context = ObserverContext(
  #     observer_type=codes.DCM.Device,
  #     observer_identifying_attributes=DeviceObserverIdentifyingAttributes(
  #         uid=generate_uid(), name="BodyPartRegression"
  #     )
  observer_device_context = ObserverContext(
    observer_type=codes.DCM.Device,
    observer_identifying_attributes=DeviceObserverIdentifyingAttributes(
        uid=generate_uid(), name="BodyPartRegression", 
        model_name = bpr_revision_number
    )
  )
  observation_context = ObservationContext(
      #observer_person_context=observer_person_context,
      observer_device_context=observer_device_context,
  )

  imaging_measurements = []
  evidence = []

  tracking_uid = generate_uid()

  qualitative_evaluations = []

  print('num_slices: ' + str(num_slices))


  #----------- Per slice ---------#

  for n in range(0,num_slices):

    slice_region = regions[n]

    # qualitative_evaluations = convert_regions_list_to_qualitative_evaluations(slice_region)

    # ----- per region ---- # 
    qualitative_evaluations = [] 
    # num_regions = len(regions)
    num_regions = len(slice_region)

    # for region in regions: 
    for region in slice_region: 
      row = bpr_regions_df.loc[bpr_regions_df['BPR_code_region'] == region]
      qualitative_evaluations.append(
          QualitativeEvaluation(
              CodedConcept(
                            value=str(row["target_CodeValue"].values[0]),
                            meaning=str(row["target_CodeMeaning"].values[0]).replace(u'\xa0', u' '),
                            # meaning = "Target Region",
                            scheme_designator=str(row["target_CodingSchemeDesignator"].values[0])
                            ), 
              CodedConcept(
                            value=str(row["CodeValue"].values[0]),
                            meaning=str(row["CodeMeaning"].values[0]),
                            scheme_designator=str(row["CodingSchemeDesignator"].values[0])
                          )
              )
          )

    # In the correct order 
    reference_dcm_file = files_sorted[n]
    image_dataset = dcmread(reference_dcm_file)
    evidence.append(image_dataset)

    src_image = hd.sr.content.SourceImageForMeasurementGroup.from_source_image(image_dataset)

    # tracking_id = "Annotations group x"
    tracking_id = "Annotations group " + str(n+1) # start indexing with 1

    measurements_group = MeasurementsAndQualitativeEvaluations(
                  tracking_identifier=TrackingIdentifier(
                      uid=tracking_uid,
                      identifier=tracking_id
                  ),
                  qualitative_evaluations=qualitative_evaluations,
                  source_images=[src_image]
              )




    imaging_measurements.append(
      measurements_group
            )
    
  #-------------------#
    
  measurement_report = MeasurementReport(
      observation_context=observation_context,
      procedure_reported=procedure_code,
      imaging_measurements=imaging_measurements
  )

  # Create the Structured Report instance
  series_instance_uid = generate_uid()
  sr_dataset = Comprehensive3DSR(
      evidence=evidence,
      content=measurement_report[0],
      series_number=100,
      series_instance_uid=series_instance_uid,
      sop_instance_uid=generate_uid(),
      instance_number=1,
      manufacturer='IDC',
      is_complete = True,
      is_final=True,
      series_description='BPR region annotations'
  )
  # series_description='BPR region annotations'

  pydicom.write_file(output_SR_file, sr_dataset)

  return sr_dataset



## BPR landmarks SR creation

In [ ]:
def get_indices_from_json(filename, tag_start, tag_end):

  """
  Gets the indices of the particular anatomy specified by the tag_start and 
  tag_end for a particular patient. 

  Arguments:
    filename  : required - the patient json filename 
    tag_start : required - the string for the start of the anatomical region 
    tag_end   : required - the string for the end of the anatomical region 

  Outputs:
    min_index : the minimum index in the patient coordinate system
    max_index : the maximum index in the patient coordinate system 
  """

  # These scores are the same for all patients  
  x = load_json(filename)
  start_score = x["look-up table"][tag_start]["mean"]
  end_score = x["look-up table"][tag_end]["mean"]

  # The actual indices 
  min_index, max_index = crop_scores(x["cleaned slice scores"], start_score, end_score)

  return min_index, max_index 

In [ ]:
def get_landmark_indices_from_json(filename):

  """
  Gets the indices of each landmark in the patient json filename and converts 
  it to slice indices. 

  Arguments:
    filename  : required - the patient json filename 

  Outputs:
    list of the corresponding landmark slices in the space of the patient. 
    Landmarks at the extreme ends - most inferior and most superior axial slices
    are removed. 
  """

  # Get the cleaned slice scores to determine the number of slices 
  x = load_json(filename)
  num_slices = len(x["cleaned slice scores"])

  # Get the list of landmarks 
  landmarks = list(x["look-up table"].keys())
  num_landmarks = len(landmarks)

  # Get the expected z_spacing - if less than 0, slices are in reverse order 
  valid_z_spacing = x["valid z-spacing"]

  # Get values for all tags 
  # Reorder the landmarks according to the mean values in ascending order 
  landmarks_dict_sorted = {}
  for n in range(0,num_landmarks):
    landmark = landmarks[n]
    landmarks_dict_sorted[landmark] = x["look-up table"][landmark]['mean']
  landmark_dict_sorted = dict(sorted(landmarks_dict_sorted.items(), key=lambda item: item[1]))
  landmarks = list(landmark_dict_sorted.keys())

  # Calculate the actual slice indices of each landmark 
  landmark_indices = {}
  for n in range(0,num_landmarks):
    landmark = landmarks[n]
    score = landmark_dict_sorted[landmark] # x["look-up table"][landmark]["mean"]
    min_index, max_index = crop_scores(x["cleaned slice scores"], score, score)
    if (valid_z_spacing > 0): 
      landmark_indices[landmark] = min_index 
    else: 
      landmark_indices[landmark] = num_slices - min_index 

  # Programmatically remove values from dictionary if it is at most inferior
  # or most superior slice 
  for n in range(0,num_landmarks):
    landmark = landmarks[n]
    if (landmark_indices[landmark]==0):
      del landmark_indices[landmark]
    elif (landmark_indices[landmark]==num_slices-1):
      del landmark_indices[landmark]

  return landmark_indices 


In [ ]:
def get_landmark_indices_list_in_slice(landmark_indices, slice_number):

  """
  Gets the list of landmarks that are assigned to a particular slice number.

  Arguments:
    flandmark_indices  : the dictionary holding the slice number for each 
                         landmark
    slice_number       : the particular slice to obtain the list of landmarks
                         for

  Outputs:
    list of the landmarks that correspond to the slice_number
  """

  key_list = list()
  items_list = landmark_indices.items()
  for item in items_list:
    if item[1] == slice_number:
        key_list.append(item[0])

  return key_list

def convert_landmarks_list_to_qualitative_evaluations(landmark_list,
                                                      landmarks_df):

  """
  Converts the list of landmarks to a qualitative_evaluations for the 
  structured report. 

  Arguments:
    landmark_list  : list of landmarks that correspond to a particular slice
    landmarks_df   : the dataframe holding the bpr landmarks metadata needed to
                     create the structured report

  Outputs:
    list of QualitativeEvaluation
  """


  qualitative_evaluations = [] 
  num_landmarks_in_slice = len(landmark_list)
  for landmark in landmark_list: 
    if not landmark in landmarks_df["BPR_code"].values:
      print("ERROR: Failed to map BPR landmark "+landmark)
      break
    else:
      landmark_row = landmarks_df[landmarks_df["BPR_code"] == landmark]
      # landmark_code = CodedConcept(value = str(int(landmark_row["CodeValue"].values[0])),
      #                              meaning = landmark_row["CodeMeaning"].values[0],
      #                              scheme_designator = landmark_row["CodingSchemeDesignator"].values[0])
      landmark_code = CodedConcept(value = str(landmark_row["CodeValue"].values[0].astype(np.int64)),
                                meaning = str(landmark_row["CodeMeaning"].values[0]),
                                scheme_designator = str(landmark_row["CodingSchemeDesignator"].values[0]))
      #print(landmarks_df["CodingSchemeDesignator"].values[0])
      landmark_modifier_code = None
      if not pd.isna(landmarks_df["modifier_CodeValue"].values[0]):
        # landmark_modifier_code = CodedConcept(value = str(int(landmark_row["modifier_CodeValue"].values[0])),
        #                              meaning = landmark_row["modifier_CodeMeaning"].values[0],
        #                              scheme_designator = landmark_row["modifier_CodingSchemeDesignator"].values[0])
        landmark_modifier_code = CodedConcept(value = str(landmark_row["modifier_CodeValue"].values[0].astype(np.int64)),
                                meaning = str(landmark_row["modifier_CodeMeaning"].values[0]),
                                scheme_designator = str(landmark_row["modifier_CodingSchemeDesignator"].values[0]))
        #print(landmarks_df["modifier_CodingSchemeDesignator"].values[0])

    qual = QualitativeEvaluation(CodedConcept(
                value="123014",
                meaning="Target Region",
                scheme_designator="DCM"  
                ), 
                landmark_code
              )
    
    if landmark_modifier_code is not None:
      qual_modifier = CodeContentItem(
          name=CodedConcept(
              value='106233006',
              meaning='Topographical modifier',
              scheme_designator='SCT',
          ),
          value=landmark_modifier_code,
          relationship_type=RelationshipTypeValues.HAS_CONCEPT_MOD
      )
      qual.append(qual_modifier)

    qualitative_evaluations.append(qual)

  return qualitative_evaluations 


In [ ]:
def create_structured_report_for_body_part_regression_landmarks(files, 
                                                                json_file, 
                                                                output_SR_file, 
                                                                bpr_revision_number,
                                                                landmarks_df):

  """Takes as input a set of DICOM files and the corresponding body part regression json file, 
     and writes a structured report (SR) to disk
     
  Inputs: 
    files               : list of CT dicom files 
    json_file           : the json file created from the BodyPartRegression prediction
    output_SR_file      : output filename for the structured report 
    bpr_revision_number : specific revision number of the bpr repo 
    landmarks_df        : the dataframe holding the bpr landmarks metadata needed to
                          create the structured report

  Outputs:
    writes the SR out to the output_SR_file.    
     
  """


  # ------ order the CT files according to the ImagePositionPatient and ImageOrientation ----# 

  num_files = len(files)

  pos_all = []  
  sop_all = [] 

  for n in range(0,num_files):
    # read dcm file 
    filename = files[n]
    ds = dcmread(filename)
    # print(ds)

    # get ImageOrientation (0020, 0037)
    # ImageOrientation = ds['0x0020','0x0037'].value
    ImageOrientation = ds.ImageOrientationPatient
    #ImageOrientation = ds.ImageOrientationPatient.value

    # get ImagePositionPatient (0020, 0032) 
    # ImagePositionPatient = ds['0x0020','0x0032'].value
    ImagePositionPatient = ds.ImagePositionPatient

    # calculate z value
    x_vector = ImageOrientation[0:3]
    y_vector = ImageOrientation[3:]
    z_vector = np.cross(x_vector,y_vector)

    # multiple z_vector by ImagePositionPatient
    pos = np.dot(z_vector,ImagePositionPatient)
    pos_all.append(pos)

    # get the SOPInstanceUID 
    sop = ds['0x0008', '0x0018'].value
    sop_all.append(sop)


  #----- order the SOPInstanceUID/files by z value ----# 

  sorted_ind = np.argsort(pos_all)
  pos_all_sorted = np.array(pos_all)[sorted_ind.astype(int)]
  sop_all_sorted = np.array(sop_all)[sorted_ind.astype(int)]
  files_sorted = np.array(files)[sorted_ind.astype(int)]

  #----- Get the landmarks as indices -----# 

  landmark_indices = get_landmark_indices_from_json(json_file)

  # ----- Create the structured report ----- # 

  # Create the report content

  procedure_code = CodedConcept(value="363679005", scheme_designator="SCT", 
                                meaning="Imaging procedure")

  # Describe the context of reported observations: the person that reported
  # the observations and the device that was used to make the observations
  # observer_person_context = ObserverContext(
  #     observer_type=codes.DCM.Person,
  #     observer_identifying_attributes=PersonObserverIdentifyingAttributes(
  #         name='Anonymous^Reader'
  #     )
  # )

  # observer_device_context = ObserverContext(
  #     observer_type=codes.DCM.Device,
  #     observer_identifying_attributes=DeviceObserverIdentifyingAttributes(
  #         uid=generate_uid(), name="BodyPartRegression_landmarks"
  #     )
  # )

  observer_device_context = ObserverContext(
      observer_type=codes.DCM.Device,
      observer_identifying_attributes=DeviceObserverIdentifyingAttributes(
          uid=generate_uid(), name="BodyPartRegression_landmarks", 
          model_name = bpr_revision_number
      )
  )
  # inputMetadata["observerContext"] = {
  #                   "ObserverType": "DEVICE",
  #                   "DeviceObserverName": "pyradiomics",
  #                   "DeviceObserverModelName": "v3.0.1"
  #                 }

  observation_context = ObservationContext(
      #observer_person_context=observer_person_context,
      observer_device_context=observer_device_context,
  )

  imaging_measurements = []
  evidence = []

  qualitative_evaluations = []

  tracking_uid = generate_uid()

  #------------- Per slice - only include landmarks that exist ------# 

  num_slices = len(files_sorted)
  annotation_count = 1 

  for n in range(0,num_slices):

    # find all the dictionary entries that match this slice number - returns a list of landmarks 
    landmark_indices_list = get_landmark_indices_list_in_slice(landmark_indices, n) # n = slice number 

    # Only include if there is a landmark for a slice 
    if (landmark_indices_list):
    
      # Create QualitativeEvaluations
      qualitative_evaluations = convert_landmarks_list_to_qualitative_evaluations(landmark_indices_list,
                                                                                  landmarks_df)

      # In the correct order 
      reference_dcm_file = files_sorted[n]
      image_dataset = dcmread(reference_dcm_file)
      evidence.append(image_dataset)

      src_image = hd.sr.content.SourceImageForMeasurementGroup.from_source_image(image_dataset)

      # tracking_id = "Annotations group x"
      # tracking_id = "Annotations group landmarks"
      # tracking_id = "Annotations group landmarks " + str(n+1) # start indexing with 1
      tracking_id = "Annotations group landmarks " + str(annotation_count) # start indexing with 1

      measurements_group = MeasurementsAndQualitativeEvaluations(
                    tracking_identifier=TrackingIdentifier(
                        uid=tracking_uid,
                        identifier=tracking_id
                    ),
                    qualitative_evaluations=qualitative_evaluations,
                    source_images=[src_image]
                )




      imaging_measurements.append(
        measurements_group
              )
      
      annotation_count += 1 # keep track of number of annotations
    


  #-------------------#
    
  measurement_report = MeasurementReport(
      observation_context=observation_context,
      procedure_reported=procedure_code,
      imaging_measurements=imaging_measurements
  )

  # Create the Structured Report instance
  series_instance_uid = generate_uid()
  sr_dataset = Comprehensive3DSR(
      evidence=evidence,
      content=measurement_report[0],
      series_number=101, # was 100 for regions
      series_instance_uid=series_instance_uid,
      sop_instance_uid=generate_uid(),
      instance_number=1,
      manufacturer='IDC',
      is_complete = True,
      is_final=True,
      series_description='BPR landmark annotations'
  )
  # series_description='BPR landmark annotations'

  pydicom.write_file(output_SR_file, sr_dataset)

  return sr_dataset



## nnUNet 3D shape features SR creation

In [ ]:
def get_label_and_names_from_metadata_json(dicomseg_json):

  """Returns two lists containing the label values and the corresponding
     CodeMeaning values

  Inputs: 
    dicomseg_json : metajson file

  Outputs:
    label_values  : label values from the metajson file 
    label_names   : the corresponding CodeMeaning values 

  """

  f = open(dicomseg_json)
  meta_json = json.load(f)

  print(meta_json)

  num_regions = len(meta_json['segmentAttributes'][0])
  print ('num_regions: ' + str(num_regions))

  label_values = []
  label_names = [] 
  for n in range(0,num_regions):
    # label_values.append(n)
    label_value = meta_json['segmentAttributes'][0][n]['labelID']
    label_name = meta_json['segmentAttributes'][0][n]['SegmentedPropertyTypeCodeSequence']['CodeMeaning']
    label_values.append(label_value)
    label_names.append(label_name)

  return label_values, label_names 

In [ ]:

def split_nii(input_file, output_directory, label_names):

  """Function to split a single multilabel nii into individual nii files. Used
     for pyradiomics feature extraction. 

  Inputs: 
    input_file       : input multi-label nii file 
    output_directory : where to save the individual nii segments 
    label_names      : the names of the labels that correspond to the order of 
                       the values in the nii input_file 

  Outputs:
    saves the individual nii files to the output_directory 
    
  """

  if not os.path.isdir(output_directory):
    os.mkdir(output_directory)

  # save with the values in the files 
  nii = nib.load(input_file)
  header = nii.header 
  img = nii.get_fdata() 
  unique_labels = list(np.unique(img))
  unique_labels.remove(0) # remove the background 

  # split and save 
  num_labels = len(unique_labels)
  for n in range(0,num_labels):
    ind = np.where(img==unique_labels[n])
    vol = np.zeros((img.shape))
    vol[ind] = 1
    new_img = nib.Nifti1Image(vol, nii.affine, nii.header)
    output_filename = os.path.join(output_directory, label_names[n] + '.nii.gz')
    nib.save(new_img, output_filename)

  return 

In [ ]:
def compute_pyradiomics_3D_features(ct_nifti_path, 
                                    label_values, 
                                    label_names, 
                                    split_pred_nifti_path, 
                                    nnunet_shape_features_code_mapping_df):

  """Function to compute pyradiomics 3D features for each label in a nifti file. 
     A single csv file is written out to disk. 

  Inputs: 
    ct_nifti_path            : the CT nifti file 
    label_values             : the label value for each of the segments from the json file 
    label_names              : the corresponding label name for each of the segments 
    split_pred_nifti_path    : where to save the individual nii segments needed 
                               for pyradiomics
    nnunet_shape_features_code_mapping_df : the df where we will obtain the 
                                            list of the shape features to 
                                            compute

  Outputs:
    Writes the features_csv_path_nnunet to disk. 
    
  """

  # Get the names of the features from the nnunet_shape_features_code_mapping_df
  shape_features = list(nnunet_shape_features_code_mapping_df['shape_feature'].values)

  # Instantiate the extractor and modify the settings to keep the 3D shape features
  extractor = featureextractor.RadiomicsFeatureExtractor()
  extractor.settings['minimumROIDimensions'] = 3 
  extractor.disableAllFeatures()
  extractor.enableFeaturesByName(shape=shape_features) 

  # Calculate features for each label and create a dataframe
  num_labels = len([f for f in os.listdir(split_pred_nifti_path) if f.endswith('.nii.gz')])
  df_list = [] 
  for n in range(0,num_labels):
    mask_path = os.path.join(split_pred_nifti_path, label_names[n] + '.nii.gz')
    # Run the extractor 
    result = extractor.execute(ct_nifti_path, mask_path) # dictionary
    # keep only the features we want
    # Get the corresponding label number -- all might not be present 
    corresponding_label_value = label_values[label_names.index(label_names[n])] 
    dict_keep = {'ReferencedSegment': corresponding_label_value, 
                 'label_name': label_names[n]}
    keys_keep = [f for f in result.keys() if 'original_shape' in f]
    # Just keep the feature keys we want
    dict_keep_new_values = {key_keep: result[key_keep] for key_keep in keys_keep}
    dict_keep.update(dict_keep_new_values)
    df1 = pd.DataFrame([dict_keep])
    # change values of columns to remove original_shape_
    df1.columns = df1.columns.str.replace('original_shape_', '')
    # Append to the ReferencedSegment and label_name df 
    df_list.append(df1)

  # concat all label features 
  df = pd.concat(df_list)

  return df

In [ ]:
def order_dicom_files_image_position(dcm_directory):
  """
  Orders the dicom files according to image position and orientation. 

  Arguments:
    dcm_directory : input directory of dcm files to put in order 

  Outputs:
    files_sorted   : dcm files in sorted order 
    sop_all_sorted : the SOPInstanceUIDs in sorted order 
    pos_all_sorted : the image position in sorted order 

  """
  files = [os.path.join(dcm_directory,f) for f in os.listdir(dcm_directory)]

  num_files = len(files)

  pos_all = []  
  sop_all = [] 

  for n in range(0,num_files):
    # read dcm file 
    filename = files[n]
    ds = dcmread(filename)

    # get ImageOrientation (0020, 0037)
    # ImageOrientation = ds['0x0020','0x0037'].value
    ImageOrientation = ds.ImageOrientationPatient

    # get ImagePositionPatient (0020, 0032) 
    # ImagePositionPatient = ds['0x0020','0x0032'].value
    ImagePositionPatient = ds.ImagePositionPatient

    # calculate z value
    x_vector = ImageOrientation[0:3]
    y_vector = ImageOrientation[3:]
    z_vector = np.cross(x_vector,y_vector)

    # multiple z_vector by ImagePositionPatient
    pos = np.dot(z_vector,ImagePositionPatient)
    pos_all.append(pos)

    # get the SOPInstanceUID 
    # sop = ds['0x0008', '0x0018'].value
    sop = ds.SOPInstanceUID
    sop_all.append(sop)


  #----- order the SOPInstanceUID/files by z value ----# 

  sorted_ind = np.argsort(pos_all)
  pos_all_sorted = np.array(pos_all)[sorted_ind.astype(int)]
  sop_all_sorted = np.array(sop_all)[sorted_ind.astype(int)]
  files_sorted = np.array(files)[sorted_ind.astype(int)]

  return files_sorted, sop_all_sorted, pos_all_sorted 

In [ ]:
def create_structured_report_metajson_for_shape_features(SeriesInstanceUID, 
                                                         SOPInstanceUID_seg,
                                                         seg_file, 
                                                         dcm_directory, 
                                                         segments_code_mapping_df,
                                                         shape_features_code_mapping_df,
                                                         df_features, 
                                                         ):
  
  """Function that creates the metajson necessary for the creation of a
  structured report from a pandas dataframe of label names and features for 
  each. 

  Inputs: 
    SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT 
                                      file 
    SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file 
    seg_file                        : filename of SEG DCM file 
    dcm_directory                   : ct directory that will be sorted in 
                                      terms of axial ordering according to the 
                                      ImagePositionPatient and ImageOrientation 
                                      fields
    segments_code_mapping_df        : dataframe that holds the names of the 
                                      segments and the associated code values etc.
    shape_features_code_mapping_df  : dataframe that holds the names of the 
                                      features and the associated code values etc. 
    df_features                     : a pandas dataframe holding the segments and a 
                                      set of 3D shape features for each 

  Outputs:
    Returns the metajson for the structured report that will then be used by
    dcmqi tid1500writer to create a structured report 
  """ 

  # --- Get the version number for the pyradiomics package --- #

  pyradiomics_version_number = str(radiomics.__version__)
  
  # --- Sort the dcm files first according to --- # 
  # --- ImagePositionPatient and ImageOrientation --- #

  files_sorted, sop_all_sorted, pos_all_sorted = order_dicom_files_image_position(dcm_directory)
  files_sorted = [os.path.basename(f) for f in files_sorted]

  # --- Create the header for the json --- # 
  
  inputMetadata = {}
  inputMetadata["@schema"]= "https://raw.githubusercontent.com/qiicr/dcmqi/master/doc/schemas/sr-tid1500-schema.json#"
  inputMetadata["SeriesDescription"] = "Measurements"
  inputMetadata["SeriesNumber"] = "1001"
  inputMetadata["InstanceNumber"] = "1"

  inputMetadata["compositeContext"] = [seg_file] # not full path

  inputMetadata["imageLibrary"] = files_sorted # not full path 

  # inputMetadata["observerContext"] = {
  #                                     "ObserverType": "PERSON",
  #                                     "PersonObserverName": "Reader1"
  #                                   }
  # inputMetadata["observerContext"] = {
  #                     "ObserverType": "DEVICE",
  #                     "DeviceObserverName": "pyradiomics",
  #                     "DeviceObserverModelName": "v3.0.1"
  #                   }
  inputMetadata["observerContext"] = {
                      "ObserverType": "DEVICE",
                      "DeviceObserverName": "pyradiomics",
                      "DeviceObserverModelName": pyradiomics_version_number
                    }

  inputMetadata["VerificationFlag"]  = "UNVERIFIED"
  inputMetadata["CompletionFlag"] =  "COMPLETE"
  inputMetadata["activitySession"] = "1"
  inputMetadata["timePoint"] = "1"

  # ------------------------------------------------------------------------- # 
  # --- Create the measurement_dict for each segment - holds all features --- # 

  measurement = [] 

  # --- Now create the dict for all features and all segments --- #

  # --- Loop over the number of segments --- #

  # number of rows in the df_features 
  num_segments = df_features.shape[0]

  # Array of dictionaries - one dictionary for each segment 
  measurement_across_segments_combined = [] 

  for segment_id in range(0,num_segments):

    ReferencedSegment = df_features['ReferencedSegment'].values[segment_id]
    FindingSite = df_features['label_name'].values[segment_id]

    print('segment_id: ' + str(segment_id))
    print('ReferencedSegment: ' + str(ReferencedSegment))
    print('FindingSite: ' + str(FindingSite))

    # --- Create the dict for the Measurements group --- # 
    TrackingIdentifier = "Measurements group " + str(ReferencedSegment)

    segment_row = segments_code_mapping_df[segments_code_mapping_df["segment"] == FindingSite]
    # print(segment_row)
        
    my_dict = {
      "TrackingIdentifier": str(TrackingIdentifier),
      "ReferencedSegment": int(ReferencedSegment),
      "SourceSeriesForImageSegmentation": str(SeriesInstanceUID),
      "segmentationSOPInstanceUID": str(SOPInstanceUID_seg),
      "Finding": {
        "CodeValue": "113343008",
        "CodingSchemeDesignator": "SCT",
        "CodeMeaning": "Organ"
      }, 
      "FindingSite": {
        "CodeValue": str(segment_row["FindingSite_CodeValue"].values[0]),
        "CodingSchemeDesignator": str(segment_row["FindingSite_CodingSchemeDesignator"].values[0]),
        "CodeMeaning": str(segment_row["FindingSite_CodeMeaning"].values[0])
      }
    }

    measurement = []  
    # number of features - number of columns in df_features - 2 (label_name and ReferencedSegment)
    num_values = len(df_features.columns)-2 

    feature_list = df_features.columns[2:] # remove first two 


    # For each measurement per region segment
    for n in range(0,num_values): 
      measurement_dict = {}
      row = df_features.loc[df_features['label_name'] == FindingSite]
      feature_row = shape_features_code_mapping_df.loc[shape_features_code_mapping_df["shape_feature"] == feature_list[n]]
      value = str(np.round(row[feature_list[n]].values[0],3))
      measurement_dict["value"] = value
      measurement_dict["quantity"] = {}
      measurement_dict["quantity"]["CodeValue"] = str(feature_row["quantity_CodeValue"].values[0])
      measurement_dict["quantity"]["CodingSchemeDesignator"] = str(feature_row["quantity_CodingSchemeDesignator"].values[0])
      measurement_dict["quantity"]["CodeMeaning"] = str(feature_row["quantity_CodeMeaning"].values[0])
      measurement_dict["units"] = {}
      measurement_dict["units"]["CodeValue"] = str(feature_row["units_CodeValue"].values[0])
      measurement_dict["units"]["CodingSchemeDesignator"] = str(feature_row["units_CodingSchemeDesignator"].values[0])
      measurement_dict["units"]["CodeMeaning"] = str(feature_row["units_CodeMeaning"].values[0])
      measurement_dict["measurementAlgorithmIdentification"] = {}
      measurement_dict["measurementAlgorithmIdentification"]["AlgorithmName"] = "pyradiomics"
      measurement_dict["measurementAlgorithmIdentification"]["AlgorithmVersion"] = str(pyradiomics_version_number)
      measurement.append(measurement_dict) 

    measurement_combined_dict = {}
    measurement_combined_dict['measurementItems'] = measurement # measurement is an array of dictionaries 

    output_dict_one_segment = {**my_dict, **measurement_combined_dict}

    # append to array for all segments 

    measurement_across_segments_combined.append(output_dict_one_segment)

  # --- Add the measurement data --- # 

  inputMetadata["Measurements"] = {}
  inputMetadata["Measurements"] = measurement_across_segments_combined

  return inputMetadata

In [ ]:
def save_structured_report_for_shape_features(SeriesInstanceUID, 
                                              SOPInstanceUID_seg, 
                                              pred_dicomseg_path, 
                                              dicomseg_json_path, 
                                              dcm_directory, 
                                              pred_nifti_path, 
                                              split_pred_nii_path, 
                                              ct_nifti_path, 
                                              segments_code_mapping_df,
                                              shape_features_code_mapping_df,
                                              sr_json_path,
                                              sr_path
                                              ):
  
  """ This function creates the SR necessary for the nnUNet shape features 

  Inputs: 
  SeriesInstanceUID               : SeriesInstanceUID of the corresponding CT 
                                    file 
  SOPInstanceUID_seg              : SOPInstanceUID of the corresponding SEG file 
  pred_dicomseg_path              : filename of DICOM SEG file 
  dicomseg_json_path              : json file for DICOM SEG file 
  dcm_directory                   : list of ct files that will be sorted in 
                                    terms of axial ordering according to the 
                                    ImagePositionPatient and ImageOrientation 
                                    fields
  pred_nifti_path                 : predictions in nifti format 
  nnunet_base_path                : path to hold the split nifti files 
  ct_nifti_path                   : filename for CT nifti file
  segments_code_mapping_df        : dataframe that holds the names of the 
                                    segments and the associated code values etc.
  shape_features_code_mapping_df  : dataframe that holds the names of the 
                                    features and the associated code values etc. 
  sr_json_path                    : the path that the metajson for the SR for 
                                    the 3D shape features will be saved 
  sr_path                         : the path that the SR for the 3D shape 
                                    features will be saved 

  Outputs:
    Returns the metajson for the structured report that will then be used by
    dcmqi tid1500writer to create a structured report 

  """

  # --- get label values and names from metajson file --- #
  label_values, label_names = get_label_and_names_from_metadata_json(dicomseg_json_path)

  # --- split the multilabel nifti into individual files --- #
  split_pred_nii_path = os.path.join(nnunet_base_path, "split_nii")
  if not os.path.isdir(split_pred_nii_path): 
    os.mkdir(split_pred_nii_path)
  split_nii(pred_nifti_path, split_pred_nii_path, label_names)

  # --- compute features and save csv for each region --- #
  if not os.path.isdir(features_csv_path_nnunet):
    os.mkdir(features_csv_path_nnunet) 
  df_features = compute_pyradiomics_3D_features(ct_nifti_path, 
                                                label_values, 
                                                label_names,
                                                split_pred_nii_path, 
                                                nnunet_shape_features_code_mapping_df)
  print ('created df_features')
  
  # --- upload csv file to bucket --- #
  # !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_features_csv_path $gs_uri_features_csv_file

  # remove nii files after saving out pyradiomics results
  !rm $split_pred_nii_path/*
  # remove csv 
  # !rm $pred_features_csv_path

  # --- Create the final metadata for the SR --- #
  inputMetadata = create_structured_report_metajson_for_shape_features(SeriesInstanceUID, 
                                                                       SOPInstanceUID_seg,
                                                                       pred_dicomseg_path, 
                                                                       dcm_directory, 
                                                                       nnunet_segments_code_mapping_df, 
                                                                       nnunet_shape_features_code_mapping_df,
                                                                       df_features)

  print ('created SR json for shape features')

  # --- Write out json --- #

  with open(sr_json_path, 'w') as f:
    json.dump(inputMetadata, f, indent=2)
  print ('wrote out json for shape features')

  # --- Save the SR for nnUNet shape features --- # 
  # inputImageLibraryDirectory = os.path.join("/content", "raw")
  # outputDICOM = os.path.join("/content","features_sr.dcm")
  # inputCompositeContextDirectory = os.path.join("/content","seg")
  inputImageLibraryDirectory = dcm_directory
  # outputDICOM = sr_json_path
  outputDICOM = sr_path
  # the name of the folder where the seg files are located 
  inputCompositeContextDirectory = os.path.basename(pred_dicomseg_path) # might need to check this
  inputMetadata_json = sr_json_path 

  print ('inputImageLibraryDirectory: ' + str(inputImageLibraryDirectory))
  print ('outputDICOM: ' + str(outputDICOM))
  print ('inputCompositeContextDirectory: ' + str(inputCompositeContextDirectory))
  print ('inputMetadata_json: ' + str(inputMetadata_json)) 
  !tid1500writer --inputImageLibraryDirectory $inputImageLibraryDirectory \
                --outputDICOM $outputDICOM  \
                --inputCompositeContextDirectory $inputCompositeContextDirectory \
                --inputMetadata $inputMetadata_json
  print ('wrote out SR for shape features')

  return 

# Putting everything together

### Setting up paths

Paths for nnUNet and BPR 

In [ ]:
############ nnUNet paths #############

data_base_path = "/content/data"
raw_base_path = "/content/data/raw/tmp"
sorted_base_path = "/content/data/raw/nlst/dicom" 

processed_base_path = "/content/data/processed/nlst/"
processed_nifti_path = os.path.join(processed_base_path, "nii")

processed_nrrd_path_nnunet = os.path.join(processed_base_path, "nnunet", "nrrd")
processed_dicomseg_path_nnunet = os.path.join(processed_base_path, "nnunet", "dicomseg")
processed_dicompm_path_nnunet = os.path.join(processed_base_path, "nnunet", "dicompm")
# added
sr_path_nnunet = os.path.join(processed_base_path, "nnunet", "sr")
if not os.path.isdir(sr_path_nnunet):
  os.mkdir(sr_path_nnunet)

nnunet_base_path = "/content/data/nnunet" # DK added. 
model_input_folder_nnunet = "/content/data/nnunet/model_input/"
model_output_folder_nnunet = "/content/data/nnunet/nnunet_output/"

features_csv_path_nnunet = "/content/data/nnunet/features"

bucket_output_base_uri_nnunet = os.path.join(bucket_base_uri, "nnunet", "nnunet_output", nlst_sub)



# -----------------
# nnU-Net pipeline parameters

experiment_folder_name = nnunet_model + "-tta" if use_tta == True else nnunet_model + "-no_tta"
bucket_experiment_folder_uri_nnunet = os.path.join(bucket_output_base_uri_nnunet, experiment_folder_name)

bucket_log_folder_uri_nnunet = os.path.join(bucket_experiment_folder_uri_nnunet, 'log')

bucket_nifti_folder_uri_nnunet = os.path.join(bucket_experiment_folder_uri_nnunet, 'nii')
bucket_softmax_pred_folder_uri_nnunet = os.path.join(bucket_experiment_folder_uri_nnunet, 'softmax_pred')

bucket_dicomseg_folder_uri_nnunet = os.path.join(bucket_experiment_folder_uri_nnunet, 'dicomseg')

# bucket_features_csv_folder_uri_nnunet = os.path.join(bucket_experiment_folder_uri_nnunet, 'features_csv')

bucket_sr_folder_uri_nnunet = os.path.join(bucket_experiment_folder_uri_nnunet, 'sr')

bucket_ct_folder_uri_nnunet = os.path.join(bucket_experiment_folder_uri_nnunet, 'ct_temp')

# -----------------
# save run information

yaml_fn = "run_params.yaml"
yaml_out_path = os.path.join(data_base_path, yaml_fn)

settings_dict = dict()
settings_dict["bucket"] = dict()
settings_dict["bucket"]["name"] = bucket_name
settings_dict["bucket"]["base_uri"] = bucket_base_uri
settings_dict["bucket"]["output_base_uri"] = bucket_output_base_uri_nnunet
settings_dict["bucket"]["experiment_folder_uri"] = bucket_experiment_folder_uri_nnunet
settings_dict["bucket"]["nifti_folder_uri"] = bucket_nifti_folder_uri_nnunet
settings_dict["bucket"]["softmax_pred_folder_uri"] = bucket_softmax_pred_folder_uri_nnunet
settings_dict["bucket"]["dicomseg_folder_uri"] = bucket_dicomseg_folder_uri_nnunet
settings_dict["bucket"]["log_folder_uri"] = bucket_log_folder_uri_nnunet

settings_dict["inference"] = dict()
settings_dict["inference"]["model"] = nnunet_model
settings_dict["inference"]["use_tta"] = use_tta
settings_dict["inference"]["export_prob_maps"] = export_prob_maps

with open(yaml_out_path, 'w') as fp:
  yaml.dump(settings_dict, fp, default_flow_style = False)

gs_uri_yaml_file = os.path.join(bucket_log_folder_uri_nnunet, yaml_fn)

!$s5cmd_path --endpoint-url https://storage.googleapis.com cp $yaml_out_path $gs_uri_yaml_file

# --------- 

# Get the csv files needed for the structured report comparison -- later get these from github 
# nnunet_segments_code_mapping_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/nnunet_segments_code_mapping.csv")
# nnunet_segments_code_mapping_df
# nnunet_shape_features_code_mapping_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/nnunet_shape_features_code_mapping.csv")
# nnunet_shape_features_code_mapping_df

######### BPR paths ############

processed_json_path = os.path.join(processed_base_path, "bpr", "json")

model_input_folder_bpr = "/content/data/bpr/model_input/"
model_input_folder_tmp_bpr = "/content/data/bpr/model_input_tmp"
model_output_folder_bpr = "/content/data/bpr/bpr_output/"
model_output_folder_tmp_bpr = "/content/data/bpr/bpr_output_tmp"
model_output_folder_sr_regions_bpr = "/content/data/bpr/sr_regions"
model_output_folder_sr_landmarks_bpr = "/content/data/bpr_sr_landmarks"

bucket_output_base_uri_bpr = os.path.join(bucket_base_uri, "bpr", "bpr_output", nlst_sub)

# -----------------
# bpr parameters 

bucket_log_folder_uri_bpr = os.path.join(bucket_output_base_uri_bpr, 'log')
bucket_nifti_folder_uri_bpr = os.path.join(bucket_output_base_uri_bpr, 'nii')
bucket_json_folder_uri_bpr = os.path.join(bucket_output_base_uri_bpr, 'json')
bucket_sr_regions_folder_uri_bpr = os.path.join(bucket_output_base_uri_bpr, 'sr_regions')
bucket_sr_landmarks_folder_uri_bpr = os.path.join(bucket_output_base_uri_bpr, 'sr_landmarks')

if not os.path.isdir(model_output_folder_sr_regions_bpr):
  os.mkdir(model_output_folder_sr_regions_bpr)
if not os.path.isdir(model_output_folder_sr_landmarks_bpr):
  os.mkdir(model_output_folder_sr_landmarks_bpr)



cp /content/data/run_params.yaml s3://idc-medima-paper-dk/nnunet/nnunet_output/nlst_gcp_vm_test_all_run/3d_fullres-tta/log/run_params.yaml


## Create the cohort using Big Query

The query uses many criteria to filter out series that cannot be processed by nnUNet or BPR. For instance, we filter according to a range of values of SliceThickness, and make sure to eliminate series that have multiple image orientations

One series is then picked per study. 

Read the query from the text file.

In [ ]:
query_text_file = '/content/queries/query.txt' 
with open(query_text_file) as f:
  query = f.read()
print(query)

WITH
  nlst_positive AS (
  SELECT
    SAFE_CAST(pid AS STRING) AS pid,
    PatientID, 
    StudyInstanceUID,
    SeriesInstanceUID 
  FROM
    `bigquery-public-data.idc_current.nlst_prsn` AS nlst
  JOIN
    `bigquery-public-data.idc_current.dicom_all` AS dicom_all
  ON
    SAFE_CAST(nlst.pid AS STRING) = dicom_all.PatientID
  WHERE
    can_scr = 1
  ),



  nlst_instances_per_series AS (
  SELECT
    ANY_VALUE(nlst_positive.PatientID) AS PatientID,
    nlst_positive.StudyInstanceUID,
    nlst_positive.SeriesInstanceUID,
    COUNT(DISTINCT(SOPInstanceUID)) AS num_instances,
    COUNT(DISTINCT(ARRAY_TO_STRING(ImagePositionPatient,"/"))) AS position_count,
    COUNT(DISTINCT(ARRAY_TO_STRING(PixelSpacing,"/"))) AS pixel_spacing_count,
    COUNT(DISTINCT(ARRAY_TO_STRING(ImageOrientationPatient,"/"))) AS orientation_count,
    MIN(SAFE_CAST(SliceThickness AS float64)) AS min_SliceThickness,
    MAX(SAFE_CAST(SliceThickness AS float64)) AS max_SliceThickness,
    MIN(SAFE_CAST(ImagePositionP

Create the dataset in the project if it doesn't already exist.

In [ ]:
# Check if the dataset exists within the project 
dataset_exists = dataset_exists_in_project(project_name, dataset_table_id)

# If it does not exist, create the dataset 
if not dataset_exists: 
  print ('creating dataset: ' + str(dataset_table_id))
  create_dataset(project_name, dataset_table_id)
else:
  print ('dataset ' + str(dataset_table_id) + ' already exists.')

dataset dataset_nlst already exists.


If the table does not exist, create it. If it does exist, choose to overwrite it, but know that the series_ids that result for the NLST collection will be different as a series_id is chosen per study. 

In [ ]:
if (gce_vm):
  client = bigquery.Client()
else: 
  client = bigquery.Client(project=project_name)

table_id = '.'.join([project_name, dataset_table_id, table_view_id_name])
job_config = bigquery.QueryJobConfig(destination=table_id)

# Try to create table 
try: 
  print ('trying to create table: ' + str(table_id))
  query_job = client.query(query, job_config=job_config) 
  result = query_job.result()  
  # cohort_df_all = result.to_dataframe(create_bqstorage_client=True)
  print ('created table: ' + str(table_id))
# If table already exists, don't do anything
except: 
  print('table ' + str(table_id) + ' already exists')
  pass 

# Overwrite the table by deleting it and creating a new one 
# Potentially dangerous. 
# That's why query should ideally only be run once, and the same table loaded each time. 
if (overwrite_view):
  print ('Overwrite table set to 1, so deleting and overwriting table: ' + str(table_id))
  # delete table 
  if (gce_vm):
    client = bigquery.Client()
  else: 
    client = bigquery.Client(project=project_name)
  client.delete_table(table_id, not_found_ok=True)  
  # create table 
  if (gce_vm):
    client = bigquery.Client()
  else: 
    client = bigquery.Client(project=project_name)
  query_job = client.query(query, job_config=job_config) 
  result = query_job.result() 
  # cohort_df_all = result.to_dataframe(create_bqstorage_client=True)


trying to create table: idc-external-018.dataset_nlst.nlst_table
table idc-external-018.dataset_nlst.nlst_table already exists


Get the list of series_id we want to process 

In [ ]:
table_id = '.'.join([project_name, dataset_table_id, table_view_id_name])

if (gce_vm):
  client = bigquery.Client()
else: 
  client = bigquery.Client(project=project_name)

query_view = f"""
  SELECT 
    DISTINCT(SeriesInstanceUID)
  FROM
    {table_id};
  """
# LIMIT 50 OFFSET @patient_offset;

start_time = time.time()
job_config = bigquery.QueryJobConfig(query_parameters=[
                                                       bigquery.ScalarQueryParameter("patient_offset", "INTEGER", int(patient_offset))
                                                       ])
result = client.query(query_view, job_config=job_config) 
series_df = result.to_dataframe(create_bqstorage_client=True)
end_time = time.time()
print ('elapsed time: ' + str(end_time-start_time)) # 2.7 seconds. 

# series_id_list_temp = sorted(list(series_df['SeriesInstanceUID'].values))
# series_id_list = series_id_list_temp[patient_offset:patient_offset+50]
# series_id_list = [str(f) for f in series_id_list]

series_id_list = list(series_df['SeriesInstanceUID'].values)

print ('num of unique series should be 50 -- the number of unique series is: ' + str(len(set(series_id_list))))

elapsed time: 2.1778056621551514
num of unique series should be 50 -- the number of unique series is: 1040


Now get the fields we want from the list of series_ids

In [ ]:
if (gce_vm):
  client = bigquery.Client()
else:
  client = bigquery.Client(project=project_name)
  
query_view = f"""
  SELECT 
    * 
  FROM
    {table_id}
  WHERE
    SeriesInstanceUID IN UNNEST(@series_id_list);
  """

start_time = time.time()
job_config = bigquery.QueryJobConfig(query_parameters=[
                                                       bigquery.ArrayQueryParameter("series_id_list", "STRING", series_id_list)
                                                       ])
result = client.query(query_view, job_config=job_config) 
cohort_df = result.to_dataframe(create_bqstorage_client=True)
end_time = time.time()
print ('elapsed time: ' + str(end_time-start_time)) #  8.6 seconds.

elapsed time: 4.368326425552368


In [ ]:
len(list(set(list(cohort_df['SeriesInstanceUID'].values))))

1040

## Optional GCP Healthcare steps 

The following steps are optional if you would like to visually assess the nnUNet and BPR results. 

One way to visually examine that the results of the nnUNet segmentation look reasonable and that BPR is producing the accurate body part predictions per slice is to set up our own instance of the OHIF viewer. If you have set up your own viewer using the steps outlined in the beginning of the notebook, perform the steps below to view your results. 

To do so, the following steps must first be performed: 
1. We first need to create a dataset, which will contain the DICOM datastore to hold our CT, SEG and SR files. 
2. We will then create a DICOM datastore within the dataset. 
3. We then later need to copy the CT, SEG, and SR files from our previously populated bucket to the DICOM datastore. 


In [ ]:
if my_ohif_app is not None: 
  # Create a dataset if it doesn't already exist 
  create_gcp_healthcare_dataset(project_name, location_id, dataset_id)
  # Create a DICOM datastore if it doesn't already exist
  create_gcp_healthcare_dicom_datastore(project_name, location_id, dataset_id, datastore_id)
  # create_gcp_healthcare_dicom_datastore(project_name, location_id, dataset_id, datastore_id_with_bpr_regions)
  # create_gcp_healthcare_dicom_datastore(project_name, location_id, dataset_id, datastore_id_with_bpr_landmarks)

datasets that exist for project_id idc-external-018, location us-central1: ['SlicerDICOMWeb_test', 'bpr_dataset', 'bpr_dataset_sr', 'bpr_pw37_dataset', 'dataset_nlst', 'mpreview_dataset', 'mpreview_dataset_ncigt', 'mpreview_dataset_prostatex', 'nnunet_seg_test01', 'project-week', 'project-week-dataset', 'siim_cmimi22_dataset']
datasets that exist for project_id idc-external-018, location us-central1: ['SlicerDICOMWeb_test', 'bpr_dataset', 'bpr_dataset_sr', 'bpr_pw37_dataset', 'dataset_nlst', 'mpreview_dataset', 'mpreview_dataset_ncigt', 'mpreview_dataset_prostatex', 'nnunet_seg_test01', 'project-week', 'project-week-dataset', 'siim_cmimi22_dataset']
datastores that exist for project_id idc-external-018, location us-central1, dataset dataset_nlst: ['2d-tta-seg-only', 'bpr_pw37_datastore', 'datastore_nlst_25_series_06_15_22', 'datastore_nlst_25_series_06_16_22', 'datastore_nlst_25_series_gcp_vm', 'datastore_nlst_25_series_gcp_vm_no_resampling', 'datastore_nlst_50_series_gcp_vm_no_resampl

In [ ]:
# delete dicom store 
# !gcloud healthcare dicom-stores delete $datastore_id --project $project_name --dataset $dataset_id --format="value(ID)" 

## Set the run parameters 

We want to create tables to keep track of the inference and total times for the nnUNet and BPR processing. To make it organized, we will create an empty table with the column names we want (schema). 

In [ ]:
# nnuNet table creation 
# If table of inference/total time results doesn't exist, create the schema based on the above 

from google.cloud.exceptions import NotFound
if (gce_vm):
  client = bigquery.Client()
else:
  client = bigquery.Client(project=project_name)

try:
  client.get_table(nnunet_table_id_fullname)
  nnunet_table_exists = 1 
except NotFound: 
  nnunet_table_exists = 0 

if (nnunet_table_exists==0):

  schema = [
      bigquery.SchemaField("PatientID", "STRING", mode="REQUIRED"), 
      bigquery.SchemaField("StudyInstanceUID", "STRING", mode="REQUIRED"), 
      bigquery.SchemaField("SeriesInstanceUID", "STRING", mode="REQUIRED"), 
      bigquery.SchemaField("num_instances", "INTEGER", mode="REQUIRED"), 
      bigquery.SchemaField("inference_time", "FLOAT", mode="REQUIRED"), 
      bigquery.SchemaField("total_time", "FLOAT", mode="REQUIRED"),
      bigquery.SchemaField("model", "STRING", mode="REQUIRED"), 
      bigquery.SchemaField("use_tta", "BOOLEAN", mode="REQUIRED"),
  ] 
  if (my_ohif_app):
    schema.append(bigquery.SchemaField('ohif_viewer_url', "STRING"))

  table = bigquery.Table(nnunet_table_id_fullname, schema=schema)
  table = client.create_table(table)  # Make an API request.
  print(
      "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
  )

else: 
  print ("Table " + str(nnunet_table_id_fullname) + ' already exists.')


Created table idc-external-018.dataset_nlst.nlst_nnunet_time_gcp_vm_test_all_run


We will do the same to hold the BPR results. 

In [ ]:
# bpr table creation
# If table of inference/total time results doesn't exist, create the schema
from google.cloud.exceptions import NotFound

if (gce_vm):
  client = bigquery.Client()
else:
  client = bigquery.Client(project=project_name)

try:
  client.get_table(bpr_table_id_fullname)
  bpr_table_exists = 1 
except NotFound: 
  bpr_table_exists = 0 

if (bpr_table_exists==0):

  schema = [
      bigquery.SchemaField("PatientID", "STRING", mode="REQUIRED"), 
      bigquery.SchemaField("StudyInstanceUID", "STRING", mode="REQUIRED"), 
      bigquery.SchemaField("SeriesInstanceUID", "STRING", mode="REQUIRED"), 
      bigquery.SchemaField("num_instances", "INTEGER", mode="REQUIRED"), 
      bigquery.SchemaField('inference_time_bpr', "FLOAT", mode="REQUIRED"),
      bigquery.SchemaField('total_time_bpr', "FLOAT", mode="REQUIRED")
  ] 
  if (my_ohif_app):
    schema.append(bigquery.SchemaField('ohif_viewer_url', "STRING"))
  table = bigquery.Table(bpr_table_id_fullname, schema=schema)
  table = client.create_table(table)  # Make an API request.
  print(
      "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
  )

else: 
  print ("Table " + str(bpr_table_id_fullname) + ' already exists.')

Created table idc-external-018.dataset_nlst.nlst_bpr_time_gcp_vm_test_all_run


Now we calculate the series that the analysis should be run for. 

In [ ]:
# exclude from processing all the patients for which a DICOM SEG object was exported already
# (stored in the specified Google Cloud Storage Bucket)
dicomseg_bucket_list = listdir_bucket_s3(project_name = project_name, # or gcs.listdir_bucket
                                          bucket_name = bucket_name,
                                          dir_gs_uri = bucket_dicomseg_folder_uri_nnunet)
already_processed_id_list = [f.split("_SEG")[0] for f in dicomseg_bucket_list if f.endswith("_SEG.dcm")]

print("\nFound %g series already processed."%(len(already_processed_id_list)))

series_to_process_id_list = sorted(list(set(series_id_list) - set(already_processed_id_list)))

print("Moving on with the remaining %g..."%(len(series_to_process_id_list)))

Getting the list of files at `s3://idc-medima-paper-dk/nnunet/nnunet_output/nlst_gcp_vm_test_all_run/3d_fullres-tta/dicomseg`...

Found 0 series already processed.
Moving on with the remaining 1040...


## Running the Per-series Analysis

Now we will run the analysis over each SeriesInstanceUID. 

In [ ]:
for idx, series_id in enumerate(series_to_process_id_list):


  #  -----------------
  # init

  start_total_nnunet = time.time()

  # init every single time, as the most recent logs are loaded from the bucket
  inference_time_dict_nnunet = dict()
  total_time_dict_nnunet = dict()
  inference_time_dict_bpr = dict()
  total_time_dict_bpr = dict()

  # set up processing flags? - discuss with Dennis 

  clear_output(wait = True)

  print("(%g/%g) Processing series %s"%(idx + 1, len(series_to_process_id_list), series_id))

  # Get the dataframe of the series being analyzed 
  series_df = cohort_df[cohort_df["SeriesInstanceUID"] == series_id]
  num_instances = series_df['num_instances'].to_list()[0]

  # Get the corresponding PatientId 
  patient_id = np.unique(series_df[series_df['Modality'] == "CT"]["PatientID"].values)
  assert len(patient_id) == 1 # sanity check - each PatientID should be unique 
  patient_id = patient_id[0] 
  print('patient_id: ' + str(patient_id))

  # Get the corresponding StudyInstanceUID
  study_id = np.unique(series_df[series_df['Modality'] == "CT"]["StudyInstanceUID"].values)
  assert len(study_id) == 1 # sanity check - each StudyInstanceUID should be unique
  study_id = study_id[0] 
  print('study_id: ' + str(study_id))

  dicomseg_fn = series_id + "_SEG.dcm"

  input_nifti_fn = series_id + "_0000.nii.gz"
  input_nifti_path = os.path.join(model_input_folder_nnunet, input_nifti_fn)

  pred_nifti_fn = series_id + ".nii.gz"
  pred_nifti_path = os.path.join(model_output_folder_nnunet, pred_nifti_fn)

  pred_softmax_folder_name = "pred_softmax"
  pred_softmax_folder_path = os.path.join(processed_nrrd_path_nnunet, series_id, pred_softmax_folder_name)

  pred_features_csv_fn = series_id + ".csv"
  pred_features_csv_path = os.path.join(features_csv_path_nnunet, pred_features_csv_fn)

  sr_fn = series_id + '_SR.dcm'
  sr_path = os.path.join(sr_path_nnunet, sr_fn)

  sr_json_fn = series_id + '_SR.json'
  sr_json_path = os.path.join(sr_path_nnunet, sr_json_fn)

  # -----------------
  # GS URI definition

  # gs URI at which the *nii.gz object is or will be stored in the bucket
  gs_uri_nifti_file = os.path.join(bucket_nifti_folder_uri_nnunet, pred_nifti_fn)

  # gs URI at which the folder storing the *.nrrd softmax probabilities is or will be stored in the bucket
  gs_uri_softmax_pred_folder = os.path.join(bucket_softmax_pred_folder_uri_nnunet, series_id)

  # gs URI at which the DICOM SEG object is or will be stored in the bucket
  gs_uri_dicomseg_file = os.path.join(bucket_dicomseg_folder_uri_nnunet, dicomseg_fn)

  # DK added - gs URI at which the CT to nii file is or will be stored in the bucket 
  gs_uri_ct_nifti_file = os.path.join(bucket_dicomseg_folder_uri_nnunet, pred_nifti_fn)

  # DK added - gs URI at which the features csv is saved if a 3d model is run 
  # gs_uri_features_csv_file = os.path.join(bucket_features_csv_folder_uri_nnunet, pred_features_csv_fn)
  # gs URI at which the DICOM SR ojbect for the shape features is or will be stored in the bucket 
  gs_uri_sr_file = os.path.join(bucket_sr_folder_uri_nnunet, sr_fn)

  # -----------------
  # preprocessing

  # Download the DICOM data 
  download_path = os.path.join(sorted_base_path, series_id) # should be deleted after bpr 
  if not os.path.exists(download_path):
    start_time_download_series_data = time.time()
    download_series_data_s5cmd(raw_base_path = raw_base_path, # --> ADD THIS TO GIT REPO. 
                               sorted_base_path = sorted_base_path,
                               series_df = series_df,
                               remove_raw = True)
    elapsed_time_download_series_data = time.time()-start_time_download_series_data

  # # DICOM CT to NIfTI - required for the processing
  # start_time_ct_to_nii = time.time()
  # preprocessing.pypla_dicom_ct_to_nifti(sorted_base_path = sorted_base_path,
  #                                       processed_nifti_path = processed_nifti_path,
  #                                       pat_id = series_id, 
  #                                       verbose = True)
  # elapsed_time_ct_to_nii = time.time()-start_time_ct_to_nii 

  # DICOM CT to NifTI - required for processing 
  start_time_ct_to_nii = time.time()
  success = dcm2niix_dicom_ct_to_nifti(sorted_base_path = sorted_base_path,
                                       processed_nifti_path = processed_nifti_path,
                                       pat_id = series_id)
  elapsed_time_ct_to_nii = time.time()-start_time_ct_to_nii 
  if success == -1:
    print("Cannot convert DICOM to NifTI using dcm2niix, either created no nii volumes or multiple volumes - stopping processing. ")
    # create file in the log directory 
    dcm2nii_log_path = os.path.join(processed_base_path, 'dcm2nii_log.txt')
    gs_uri_dcm2nii_log = os.path.join(bucket_log_folder_uri_nnunet, series_id + '_dcm2nii_log.txt')
    with open(dcm2nii_log_path, 'w') as f:
      f.write("Cannot convert DICOM to NifTI using dcm2niix, either created no nii volumes or multiple volumes - stopping processing. ")
    f.close()
    # !$s5cmd_path cp $dcm2nii_log_path $gs_uri_dcm2nii_log
    !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $dcm2nii_log_path $gs_uri_dcm2nii_log
    continue 

  # upload nifti file to bucket 
  ct_nifti_path = os.path.join(processed_nifti_path,series_id,series_id+"_CT.nii.gz")
  !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $ct_nifti_path $gs_uri_ct_nifti_file


  # prepare the `model_input` folder for the inference phase
  preprocessing.prep_input_data(processed_nifti_path = processed_nifti_path,
                                model_input_folder = model_input_folder_nnunet,
                                pat_id = series_id)
  
  start_inference_nnunet = time.time()
  # run the DL-based prediction
  processing.process_patient_nnunet(model_input_folder = model_input_folder_nnunet,
                                    model_output_folder = model_output_folder_nnunet, 
                                    nnunet_model = nnunet_model, 
                                    use_tta = use_tta,
                                    export_prob_maps = export_prob_maps)
  elapsed_inference_nnunet = time.time() - start_inference_nnunet

  if export_prob_maps:
    # convert the softmax predictions to NRRD files
    postprocessing.numpy_to_nrrd(model_output_folder = model_output_folder_nnunet,
                                processed_nrrd_path = processed_nrrd_path_nnunet,
                                pat_id = series_id,
                                output_folder_name = pred_softmax_folder_name)

    # copy the nnU-Net *.npz softmax probabilities in the chosen bucket
    !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_softmax_folder_path/ $gs_uri_softmax_pred_folder/

  # copy the nnU-Net *.nii.gz binary masks in the chosen bucket --> Do we need this? 
  # !gsutil -m cp $pred_nifti_path $gs_uri_nifti_file
  !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_nifti_path $gs_uri_nifti_file

  # -----------------
  # post-processing

  # FIXME: consider removing this? (if only NIfTIs will be used to produce the DICOM SEGs)
  # mandatory post-processing to convert the NIfTI file from the pipeline
  # to a NRRD file (same content)
  if not os.path.isdir(os.path.join(processed_nrrd_path_nnunet,series_id)):
    os.mkdir(os.path.join(processed_nrrd_path_nnunet,series_id))
  postprocessing.pypla_postprocess(processed_nrrd_path = processed_nrrd_path_nnunet,
                                  model_output_folder = model_output_folder_nnunet,
                                  pat_id = series_id)

  # Modify the dicomseg_json file so that the SegmentAlgorithmName is representative of the model and other parameters 
  # Writes out the json file 
  SegmentAlgorithmName = experiment_folder_name 
  dicomseg_json_path_modified = "/content/data/dicomseg_metadata_" + SegmentAlgorithmName + '.json'
  modify_dicomseg_json_file(dicomseg_json_path, dicomseg_json_path_modified, SegmentAlgorithmName)
  # upload the json file 
  gs_uri_dicomseg_json_file = os.path.join(bucket_experiment_folder_uri_nnunet, 'dicomseg_metadata_' + SegmentAlgorithmName + '.json')
  !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $dicomseg_json_path_modified $gs_uri_dicomseg_json_file

  # convert the NRRD file to a DICOM SEG object
  postprocessing.nrrd_to_dicomseg(sorted_base_path = sorted_base_path,
                                  processed_base_path = processed_base_path + "nnunet",
                                  dicomseg_json_path = dicomseg_json_path,
                                  pat_id = series_id)
  
  pred_dicomseg_path = os.path.join(processed_dicomseg_path_nnunet, series_id, dicomseg_fn)
  !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_dicomseg_path $gs_uri_dicomseg_file

  # -----------------
  # extract features if nnunet_model is 3d and save structured report 
  if ('3d' in nnunet_model):
    seg_dcm = pydicom.dcmread(pred_dicomseg_path)
    SOPInstanceUID_seg = seg_dcm.file_meta['0x0002', '0x0003'].value
    dcm_directory = os.path.join(sorted_base_path, series_id, 'CT')
    nnunet_features_metajson = save_structured_report_for_shape_features(series_id, 
                                                                         SOPInstanceUID_seg, 
                                                                         pred_dicomseg_path,  
                                                                         dicomseg_json_path, 
                                                                         dcm_directory,
                                                                         pred_nifti_path, 
                                                                         nnunet_base_path, 
                                                                         ct_nifti_path, 
                                                                         nnunet_segments_code_mapping_df,
                                                                         nnunet_shape_features_code_mapping_df,
                                                                         sr_json_path,
                                                                         sr_path
                                                                         )
    # Copy SR to bucket 
    !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $sr_path $gs_uri_sr_file

  
  # -----------------
  # clean up files in nnunet path 
  clean_files_in_nnunet_path(series_id, processed_base_path, model_output_folder_nnunet, processed_nrrd_path_nnunet)

  # remove the NIfTI file the prediction was computed from
  !rm $input_nifti_path

  # -----------------
  # GCP healthcare steps --> MAKE SURE THIS WORKS. 
  if my_ohif_app is not None: 
    # copy SEG from bucket to the DICOM datastore 
    experiment_folder_name = nnunet_model + "-tta" if use_tta == True else nnunet_model + "-no_tta"
    bucket_experiment_folder_uri_nnunet = os.path.join(bucket_output_base_uri_nnunet, experiment_folder_name)
    bucket_dicomseg_folder_uri_nnunet = os.path.join(bucket_experiment_folder_uri_nnunet, 'dicomseg')
    print (bucket_dicomseg_folder_uri_nnunet)
    my_bucket = bucket_dicomseg_folder_uri_nnunet.split('s3://')[1]
    !gcloud healthcare dicom-stores import gcs $datastore_id \
                                                --dataset=$dataset_id \
                                                --location=$location_id \
                                                --project=$project_name \
                                                --gcs-uri=gs://$my_bucket/$dicomseg_fn
    # !gcloud healthcare dicom-stores import gcs $datastore_id_with_bpr_regions \
    #                                             --dataset=$dataset_id \
    #                                             --location=$location_id \
    #                                             --project=$project_name \
    #                                             --gcs-uri=gs://$my_bucket/$dicomseg_fn
    # !gcloud healthcare dicom-stores import gcs $datastore_id_with_bpr_landmarks \
    #                                             --dataset=$dataset_id \
    #                                             --location=$location_id \
    #                                             --project=$project_name \
    #                                             --gcs-uri=gs://$my_bucket/$dicomseg_fn
    # !gcloud healthcare dicom-stores import gcs $datastore_id \
    #                                             --dataset=$dataset_id \
    #                                             --location=$location_id \
    #                                             --project=$project_name \
    #                                             --gcs-uri=gs://$my_bucket/**.dcm
    # copy SR with shape features from bucket to the DICOM datastore 
    if ('3d' in nnunet_model):
      my_bucket = bucket_sr_folder_uri_nnunet.split('s3://')[1]
      !gcloud healthcare dicom-stores import gcs $datastore_id \
                                              --dataset=$dataset_id \
                                              --location=$location_id \
                                              --project=$project_name \
                                              --gcs-uri=gs://$my_bucket/$sr_fn
    # !gcloud healthcare dicom-stores import gcs $datastore_id_with_bpr_regions \
    #                                         --dataset=$dataset_id \
    #                                         --location=$location_id \
    #                                         --project=$project_name \
    #                                         --gcs-uri=gs://$my_bucket/$sr_fn
    # !gcloud healthcare dicom-stores import gcs $datastore_id_with_bpr_landmarks \
    #                                         --dataset=$dataset_id \
    #                                         --location=$location_id \
    #                                         --project=$project_name \
    #                                         --gcs-uri=gs://$my_bucket/$sr_fn



    # Create two server ohif_viewer_url 
    ohif_viewer_url = create_ohif_viewer_2server_url(my_ohif_app, 
                                                     project_name, 
                                                     location_id, 
                                                     dataset_id, 
                                                     datastore_id, 
                                                     study_id)

    # # Copy the raw CT to bucket 
    # raw_path = os.path.join(sorted_base_path, series_id, 'CT')
    # !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $raw_path/ $bucket_ct_folder_uri_nnunet/
    # # Copy from the bucket to the DICOM datastore
    # my_bucket = bucket_ct_folder_uri_nnunet.split('s3://')[1]
    # !gcloud healthcare dicom-stores import gcs $datastore_id \
    #                                         --dataset=$dataset_id \
    #                                         --location=$location_id \
    #                                         --project=$project_name \
    #                                         --gcs-uri=gs://$my_bucket/*.dcm
    # # Delete files from bucket 
    # # !$s5cmd_path --endpoint-url https://storage.googleapis.com rm $raw_path/
    # !$s5cmd_path --endpoint-url https://storage.googleapis.com rm $bucket_ct_folder_uri_nnunet/

    # # Create the ohif url 
    # ohif_viewer_url = create_ohif_viewer_url(my_ohif_app, 
    #                                          project_name, 
    #                                          location_id, 
    #                                          dataset_id, 
    #                                          datastore_id, 
    #                                          study_id)
    
  elapsed_total_nnunet = time.time() - start_total_nnunet

  print("End-to-end processing of %s completed in %g seconds.\n"%(series_id, elapsed_total_nnunet))

  # -----------------
  # log processing information to BigQuery tables --> MAKE SURE THIS WORKS.  
  row_to_insert = [] 
  if my_ohif_app is not None: # if using ohif viewer 
    row_to_insert_dict = OrderedDict([("PatientID", patient_id), 
                                      ("StudyInstanceUID", study_id), 
                                      ("SeriesInstanceUID", series_id), 
                                      ("num_instances", int(num_instances)), 
                                      ("inference_time", elapsed_inference_nnunet), 
                                      ("total_time", elapsed_total_nnunet), 
                                      ("model", nnunet_model), 
                                      ("use_tta", use_tta), 
                                      ("ohif_viewer_url", ohif_viewer_url)])
  else: # if not using ohif viewer 
    row_to_insert_dict = OrderedDict([("PatientID", patient_id), 
                                      ("StudyInstanceUID", study_id), 
                                      ("SeriesInstanceUID", series_id), 
                                      ("num_instances", int(num_instances)), 
                                      ("inference_time", elapsed_inference_nnunet), 
                                      ("total_time", elapsed_total_nnunet), 
                                      ("model", nnunet_model), 
                                      ("use_tta", use_tta)])
  row_to_insert.append(row_to_insert_dict)

  # append_row_to_bq_table(row_to_insert, table_ref_nnunet,
  #                        dataset_table_id, nnunet_table_id, project_name) 
  append_row_to_bq_table_with_query(project_name, 
                                    dataset_table_id, 
                                    nnunet_table_id, 
                                    series_id, 
                                    row_to_insert)


  # ------------
  # BPR 

  start_total_bpr = time.time()

  input_nifti_fn = series_id + ".nii.gz"
  input_nifti_path = os.path.join(model_input_folder_bpr, input_nifti_fn)

  pred_json_fn = series_id + ".json"
  pred_json_path = os.path.join(model_output_folder_bpr, pred_json_fn)

  sr_bpr_regions_fn = series_id + "_bpr_regions.dcm"
  sr_bpr_regions_path = os.path.join(model_output_folder_sr_regions_bpr, sr_bpr_regions_fn)

  sr_bpr_landmarks_fn = series_id + "_bpr_landmarks.dcm"
  sr_bpr_landmarks_path = os.path.join(model_output_folder_sr_landmarks_bpr, sr_bpr_landmarks_fn)

  # gs URI at which the *.json object is or will be stored in the bucket
  gs_uri_json_file = os.path.join(bucket_json_folder_uri_bpr, pred_json_fn)
  has_json_already = file_exists_in_bucket_s3(project_name = project_name,
                                              bucket_name = bucket_name,
                                              file_gs_uri = gs_uri_json_file)

  # -----------------
  # DL-inference for BPR 
  # json not found 
  if has_json_already == False: 

    # If ct dicom directory does not exist, download the ct files 
    ct_dicom_path = os.path.join(sorted_base_path, series_id) 
    if not os.path.isdir(ct_dicom_path):
      start_time_download_series_data = time.time()
      download_series_data_s5cmd(raw_base_path = raw_base_path,
                                  sorted_base_path = sorted_base_path,
                                  series_df = series_df,
                                  remove_raw = True)
      elapsed_time_download_series_data = time.time()-start_time_download_series_data

    # If ct nifti doesn't exist, create it 
    pat_dir_nifti_path = os.path.join(processed_nifti_path, series_id)
    ct_nifti_path = os.path.join(pat_dir_nifti_path, series_id + "_CT.nii.gz")
    if not os.path.exists(ct_nifti_path):
      # start_time_ct_to_nii = time.time()
      # preprocessing.pypla_dicom_ct_to_nifti(sorted_base_path = sorted_base_path,
      #                         processed_nifti_path = processed_nifti_path,
      #                         pat_id = series_id, 
      #                         verbose = True)
      # elapsed_time_ct_to_nii = time.time()-start_time_ct_to_nii 

      # DICOM CT to NifTI - required for processing 
      start_time_ct_to_nii = time.time()
      success = dcm2niix_dicom_ct_to_nifti(sorted_base_path = sorted_base_path,
                                           processed_nifti_path = processed_nifti_path,
                                           pat_id = series_id)
      elapsed_time_ct_to_nii = time.time()-start_time_ct_to_nii 
      if success == -1:
        print("Cannot convert DICOM to NifTI using dcm2niix, either created no nii volumes or multiple volumes - stopping processing. ")
        # create file in the log directory 
        dcm2nii_log_path = os.path.join(processed_base_path, 'dcm2nii_log.txt')
        gs_uri_dcm2nii_log = os.path.join(bucket_log_folder_uri_bpr, series_id + '_dcm2nii_log.txt')
        with open(dcm2nii_log_path, 'w') as f:
          f.write("Cannot convert DICOM to NifTI using dcm2niix, either created no nii volumes or multiple volumes - stopping processing. ")
        f.close()
        # !$s5cmd_path cp $dcm2nii_log_path $gs_uri_dcm2nii_log
        !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $dcm2nii_log_path $gs_uri_dcm2nii_log
        continue 

    # prepare the `model_input` folder for the inference phase
    prep_input_data_bpr(processed_nifti_path = processed_nifti_path,
                        model_input_folder = model_input_folder_bpr,
                        pat_id = series_id)

    # single patient needs a temporary folder, as bpr inference takes as input
    # a directory

    model_input_folder_series = os.path.join(model_input_folder_bpr,series_id+'.nii.gz')
    !cp $model_input_folder_series $model_input_folder_tmp_bpr

    start_inference_bpr = time.time()
    process_patient_bpr(model_input_folder = model_input_folder_tmp_bpr,
                        model_output_folder = model_output_folder_tmp_bpr,
                        model = "/content/models/bpr_model/public_bpr_model/")
    elapsed_inference_bpr = time.time() - start_inference_bpr

    !cp $model_output_folder_tmp_bpr/* $model_output_folder_bpr

    # copy the bpr *.json prediction in the chosen bucket 
    s_time = time.time()
    !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $pred_json_path $gs_uri_json_file 
    e_time = time.time()
    print ('Copy json file to bucket: ' + str(e_time-s_time))

    # ------
    # Create the SR to hold the body part prediction region values 
    dcm_files = [os.path.join(sorted_base_path,series_id,'CT',f) for f in os.listdir(os.path.join(sorted_base_path,series_id,'CT')) if f.endswith(".dcm")]
    sr = create_structured_report_for_body_part_regression_regions(dcm_files, 
                                                                   pred_json_path, 
                                                                   sr_bpr_regions_path, 
                                                                   bpr_revision_number,
                                                                   bpr_regions_df)

    # copy SR to bucket 
    gs_uri_sr_bpr_regions_file = os.path.join(bucket_sr_regions_folder_uri_bpr, sr_bpr_regions_fn)
    !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $sr_bpr_regions_path $gs_uri_sr_bpr_regions_file 

    # ------
    # Create the SR to hold the body part prediction landmark values 
    gs_uri_sr_bpr_landmarks_file = os.path.join(bucket_sr_landmarks_folder_uri_bpr, sr_bpr_landmarks_fn)
    metajson = create_structured_report_for_body_part_regression_landmarks(dcm_files, 
                                                                           pred_json_path, 
                                                                           sr_bpr_landmarks_path, 
                                                                           bpr_revision_number,
                                                                           landmarks_df)

    # copy SR to bucket 
    gs_uri_sr_bpr_landmarks_file = os.path.join(bucket_sr_landmarks_folder_uri_bpr, sr_bpr_landmarks_fn)
    !$s5cmd_path --endpoint-url https://storage.googleapis.com cp $sr_bpr_landmarks_path $gs_uri_sr_bpr_landmarks_file 


    # -----------------
    # GCP healthcare steps 
    if my_ohif_app is not None: 
      # copy SR file for bpr regions from the bucket to the DICOM datastore 
      my_bucket = bucket_sr_regions_folder_uri_bpr.split('s3://')[1]
      !gcloud healthcare dicom-stores import gcs $datastore_id \
                                                  --dataset=$dataset_id \
                                                  --location=$location_id \
                                                  --project=$project_name \
                                                  --gcs-uri=gs://$my_bucket/$sr_bpr_regions_fn
      my_bucket = bucket_sr_landmarks_folder_uri_bpr.split('s3://')[1]
      !gcloud healthcare dicom-stores import gcs $datastore_id \
                                                  --dataset=$dataset_id \
                                                  --location=$location_id \
                                                  --project=$project_name \
                                                  --gcs-uri=gs://$my_bucket/$sr_bpr_landmarks_fn
      # !gcloud healthcare dicom-stores import gcs $datastore_id_with_bpr_regions \
      #                                             --dataset=$dataset_id \
      #                                             --location=$location_id \
      #                                             --project=$project_name \
      #                                             --gcs-uri=gs://$my_bucket/$sr_bpr_regions_fn
      # # copy SR file for bpr landmarks from the bucket to the DICOM datastore 
      # my_bucket = bucket_sr_landmarks_folder_uri_bpr.split('s3://')[1]
      # !gcloud healthcare dicom-stores import gcs $datastore_id_with_bpr_landmarks \
      #                                             --dataset=$dataset_id \
      #                                             --location=$location_id \
      #                                             --project=$project_name \
      #                                             --gcs-uri=gs://$my_bucket/$sr_bpr_landmarks_fn
      # !gcloud healthcare dicom-stores import gcs $datastore_id \
      #                                             --dataset=$dataset_id \
      #                                             --location=$location_id \
      #                                             --project=$project_name \
      #                                             --gcs-uri=gs://$my_bucket/**.dcm
      # Create two server ohif_viewer_url 
      ohif_viewer_url = create_ohif_viewer_2server_url(my_ohif_app, 
                                                      project_name, 
                                                      location_id, 
                                                      dataset_id, 
                                                      datastore_id, 
                                                      study_id)
      # # Create ohif_viewer_url 
      # ohif_viewer_url = create_ohif_viewer_url(my_ohif_app, 
      #                                          project_name, 
      #                                          location_id, 
      #                                          dataset_id, 
      #                                          datastore_id, 
      #                                          study_id)

    # -----------------
    # remove files from BPR  
    clean_files_in_bpr_path(series_id, 
                            model_input_folder_bpr, 
                            model_input_folder_tmp_bpr, 
                            model_output_folder_tmp_bpr, 
                            model_output_folder_sr_regions_bpr, 
                            model_output_folder_sr_landmarks_bpr,
                            input_nifti_path, 
                            sorted_base_path, 
                            processed_nifti_path)
      
    elapsed_total_bpr = time.time() - start_total_bpr
    print("End-to-end processing of %s completed in %g seconds.\n"%(series_id, elapsed_total_bpr))

    # -----------------
    # log processing information to BigQuery tables --> MAKE SURE THIS WORKS.
    row_to_insert = [] 
    if my_ohif_app is not None: # if using ohif viewer 
      row_to_insert_dict = OrderedDict([("PatientID", patient_id),
                                        ("StudyInstanceUID", study_id), 
                                        ("SeriesInstanceUID", series_id),
                                        ("num_instances", int(num_instances)), 
                                        ("inference_time_bpr", elapsed_inference_bpr),
                                        ("total_time_bpr", elapsed_total_bpr), 
                                        ("ohif_viewer_url", ohif_viewer_url)])
    else: # if not using ohif viewer 
      row_to_insert_dict = OrderedDict([("PatientID", patient_id),
                                    ("StudyInstanceUID", study_id), 
                                    ("SeriesInstanceUID", series_id),
                                    ("num_instances", int(num_instances)), 
                                    ("inference_time_bpr", elapsed_inference_bpr),
                                    ("total_time_bpr", elapsed_total_bpr)])
    row_to_insert.append(row_to_insert_dict)

    # append_row_to_bq_table(row_to_insert, table_ref_bpr,
    #                        dataset_table_id, bpr_table_id, project_name)
    append_row_to_bq_table_with_query(project_name, 
                                      dataset_table_id, 
                                      bpr_table_id, 
                                      series_id, 
                                      row_to_insert)



Streaming output truncated to the last 5000 lines.
wrote out SR for shape features
cp /content/data/processed/nlst/nnunet/sr/1.2.840.113654.2.55.112215700690439643775188326141146413893_SR.dcm s3://idc-medima-paper-dk/nnunet/nnunet_output/nlst_gcp_vm_test_all_run/3d_fullres-tta/sr/1.2.840.113654.2.55.112215700690439643775188326141146413893_SR.dcm
rm: cannot remove '/content/data/nnunet/nnunet_output//*.npz': No such file or directory
rm: cannot remove '/content/data/processed/nlst/nnunet/nrrd/1.2.840.113654.2.55.112215700690439643775188326141146413893/pred_softmax/*.nrrd': No such file or directory
s3://idc-medima-paper-dk/nnunet/nnunet_output/nlst_gcp_vm_test_all_run/3d_fullres-tta/dicomseg
Request issued for: [datastore_nlst_gcp_vm_test_all_run]
name: projects/idc-external-018/locations/us-central1/datasets/dataset_nlst/dicomStores/datastore_nlst_gcp_vm_test_all_run
Request issued for: [datastore_nlst_gcp_vm_test_all_run]
name: projects/idc-external-018/locations/us-central1/datasets/

# DO NOT RUN. Old run over multiple models 

In [ ]:
def format_dict(input_dict):
  
  """
  Format dictionary [...]

  Arguments:
    input_dict : required - 
    
  Returns:
    output_df : 

  """

  output_df = pd.DataFrame.from_dict(data = input_dict, orient = "index")

  output_df = output_df.reset_index()
  output_df = output_df.rename(columns = {"index" : "PatientID", 
                                          0 : "inference_time"}) 
  
  return output_df

In [ ]:
# def format_dict_DK(input_dict, column_name):
  
#   """
#   Format dictionary [...]

#   Arguments:
#     input_dict : required - 
    
#   Returns:
#     output_df : 

#   """

#   output_df = pd.DataFrame.from_dict(data = input_dict, orient = "index")

#   output_df = output_df.reset_index()
#   output_df = output_df.rename(columns = {"index" : "PatientID", 
#                                           0 : column_name}) 
  
#   return output_df

def format_dict_columns(input_dict, index_name, column_name):
  
  """
  Formats the dictionary 

  Arguments:
    input_dict  : input dictionary  
    index_name  : name of index column 
    column_name : name of column with value to hold 
    
  Returns:
    output_df : formatted dataframe 

  """

  output_df = pd.DataFrame.from_dict(data = input_dict, orient = "index")

  output_df = output_df.reset_index()
  output_df = output_df.rename(columns = {"index" : index_name, 
                                          0 : column_name}) 
  
  return output_df

In [ ]:
def add_value_to_table(project_name, dataset_id, table_id, column_name, pat_id, value_name, value_to_add, dict_to_add):

  """
  This function adds a value to a table - per PatientID, SeriesInstanceUID, any other field
  First check if the table exists. If it does, check if the id exists. If the id/value exists, 
  replace with the current values. If the id does not exist, append to the table. 

  Arguments:
    project_name : name of project
    dataset_id   : name of dataset 
    table_id     : name of table
    column_name  : column name that the table will have - e.g. PatientID, SeriesInstanceUID 
    pat_id       : the patient id, series id, or any other id for which the value will be added
    value_name   : the name of the value - inference time, elapsed time 
    value_to_add : the value to add 
    dict_to_add  : the original dictionary that holds the value per id 
    
  Returns:
    

  """

  
  # ----------------- Save value in a table instead of csv  ----------- #

  ##################################### check if table exists ########################

  # Check if the table exists 
  from google.cloud.exceptions import NotFound
  table_id_fullname = '.'.join([project_name, dataset_id, table_id])
  client = bigquery.Client(project=project_name)
  try:
    client.get_table(table_id_fullname) 
    table_found = True
  except NotFound:
    table_found = False

  print ('table_found: ' + str(table_found))

  ############################################################
  ### If the table is found, check if the pat_id exists ###
  ############################################################

  if (table_found):
    # do this for now 
    if (column_name=="PatientID"):
      query = f"""
        SELECT 
          COUNT(PatientID) AS num_instances
        FROM 
          {table_id_fullname}
        WHERE 
          PatientID IN UNNEST (@pat_id_change);
      """
    elif (column_name=="SeriesInstanceUID"):
      query = f"""
        SELECT 
          COUNT(SeriesInstanceUID) AS num_instances
        FROM 
          {table_id_fullname}
        WHERE 
          SeriesInstanceUID IN UNNEST (@pat_id_change);
      """
    job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ArrayQueryParameter("pat_id_change", "STRING", [pat_id])])
    # Can't pass in the column_name like this
    # query = f"""
    #   SELECT
    #     COUNT(@column_name) AS num_instances
    #   FROM
    #     {table_id_fullname}
    #   WHERE
    #     @column_name IN UNNEST(@pat_id_change);
    #     """ 
    # job_config = bigquery.QueryJobConfig(query_parameters=[
    #                                                       bigquery.ArrayQueryParameter("pat_id_change", "STRING", [str(pat_id)]), 
    #                                                       bigquery.ScalarQueryParameter("column_name", "STRING", column_name)
    #                                                       ])
    result = client.query(query, job_config=job_config)
    df_result = result.to_dataframe()
    count = df_result['num_instances'][0]
  else: 
    count = 0 
  print ('count: ' + str(count))

  ############################################################################
  ### If the count > 0, series id exists, so replace with current time     ### 
  ### If count = 0, either series id doesn't exist or table doesn't exist, ###
  ### so create it                                                         ###
  ############################################################################

  if (count > 0): 
      client = bigquery.Client(project=project_name)
      if (column_name=="PatientID"):
        query = f"""
            UPDATE 
              {table_id_fullname}
            SET 
              time = @value_to_add
            WHERE 
              PatientID = @pat_id_change;
            """ 
      elif (column_name=="SeriesInstanceUID"):
        query = f"""
            UPDATE 
              {table_id_fullname}
            SET 
              time = @value_to_add
            WHERE 
              SeriesInstanceUID = @pat_id_change;
            """ 
      job_config = bigquery.QueryJobConfig(query_parameters=[
                                                            bigquery.ScalarQueryParameter("pat_id_change", "STRING", str(pat_id)), 
                                                            bigquery.ScalarQueryParameter("value_to_add", "FLOAT", value_to_add)
                                                           ])   
      # Did not work. 
      # query = f"""
      #   UPDATE 
      #     {table_id_fullname}
      #   SET 
      #     @value_name = @value_to_add
      #   WHERE 
      #     @column_name IN UNNEST(@pat_id_change);
      #   """
      # job_config = bigquery.QueryJobConfig(query_parameters=[
      #                                                       bigquery.ArrayQueryParameter("pat_id_change", "STRING", [str(pat_id)]), 
      #                                                       bigquery.ScalarQueryParameter(value_name, "FLOAT", value_to_add),
      #                                                       bigquery.ScalarQueryParameter("column_name", "STRING", column_name)
      #                                                       ])
      result = client.query(query, job_config=job_config)
  else: 
    # add_to_df = format_dict_DK(dict_to_add, value_name)
    add_to_df = format_dict_columns(dict_to_add, index_name=column_name, column_name=value_name) 
    table_id_fullname = '.'.join([project_name, dataset_id, table_id])
    client = bigquery.Client(project=project_name)
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
    job = client.load_table_from_dataframe(add_to_df, table_id_fullname, job_config=job_config)

  return 

In [ ]:
def save_time_and_num_instances_to_table(project_name, dataset_id, table_id, dict_to_add):

  table_id_fullname = '.'.join([project_name, dataset_id,table_id])

  add_to_df = pd.DataFrame.from_dict(data = dict_to_add, orient = "index")
  add_to_df = add_to_df.reset_index()
  # add_to_df = add_to_df.rename(columns = {"index" : 'SeriesInstanceUID', 
  #                                         0 : 'time', 
  #                                         1 : 'num_instances'})
  add_to_df = add_to_df.rename(columns = {"index" : 'SeriesInstanceUID', 
                                        0 : 'num_instances', 
                                        1 : "nnunet_inference_model_2d", 
                                        2 : "nnunet_total_time_model_2d"})

  table_id_fullname = '.'.join([project_name, dataset_id, table_id])
  client = bigquery.Client(project=project_name)
  job_config = bigquery.LoadJobConfig()
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
  job = client.load_table_from_dataframe(add_to_df, table_id_fullname, job_config=job_config)

  return

In [ ]:
def check_value_exists_in_table(table_id_fullname, project_name, series_id, field_name):
  
  """
  This function checks if a value exists in a table. 

  Arguments:
    table_id_fullname    : the full name of the table (project_name.dataset_id.table_id)
    project_name         : project name 
    series_id            : series id to check if the field exists 
    field_name           : the field name to check for the series 
    
  Returns:
    value_found          : 1 if the value is found for the field and the series_id, 0 otherwise 

  """
    
  client = bigquery.Client(project=project_name)
  # query = f"""
  #       SELECT 
  #         *
  #       FROM 
  #         {table_id_fullname}
  #       WHERE 
  #         SeriesInstanceUID = {series_id} 
  #         AND {field_name} IS NOT NULL 
  #     """
  # job_config = bigquery.QueryJobConfig()
  query = f"""
      SELECT 
        *
      FROM 
        {table_id_fullname}
      WHERE
        SeriesInstanceUID = @series_id AND 
        @field_name IS NOT NULL 
    """
  job_config = bigquery.QueryJobConfig(query_parameters = [bigquery.ScalarQueryParameter("series_id", "STRING", series_id), 
                                                          bigquery.ScalarQueryParameter("field_name", "STRING", field_name)])
  result = client.query(query, job_config=job_config)
  series = result.to_dataframe()
  
  if (len(series)==0):
    value_found = 0 
  else: 
    value_found = 1  

  return value_found

In [ ]:
def pypla_dicom_ct_to_nrrd_resample(sorted_base_path, processed_nrrd_path,
                                    pat_id, verbose = True):
  
  """
  Sorted DICOM patient data to NRRD file (CT volume).
  The nrrd volume is resampled to 1x1x2.5, from the supplementary segThor material. 
  The header of the nrrd volume is just used to create the nrrd files of the individual 
  segments, it is not used otherwise. 

  Arguments:
    sorted_base_path    : required - path to the folder where the sorted data should be stored.
    processed_nrrd_path : required - path to the folder where the preprocessed NRRD data are stored
    remove_raw          : required - patient ID (used for naming purposes).
    verbose             : optional - whether to run pyplastimatch in verbose mode. Defaults to true.
  
  Outputs:
    This function [...]
  """

  # given that everything is standardised already, compute the paths
  path_to_dicom_ct_folder = os.path.join(sorted_base_path, pat_id, "CT")
  
  # sanity check
  assert(os.path.exists(path_to_dicom_ct_folder))
  
  pat_dir_nrrd_path = os.path.join(processed_nrrd_path, pat_id)
  if not os.path.exists(pat_dir_nrrd_path):
    os.mkdir(pat_dir_nrrd_path)

  # output NRRD CT
  # ct_nrrd_path = os.path.join(pat_dir_nrrd_path, pat_id + "_CT.nrrd")
  ct_nrrd_path = os.path.join(pat_dir_nrrd_path, pat_id + "_CT_orig.nrrd")

  # logfile for the plastimatch conversion
  log_file_path = os.path.join(pat_dir_nrrd_path, pat_id + '_pypla.log')

  # DICOM CT to NRRD conversion (if the file doesn't exist yet)
  if not os.path.exists(ct_nrrd_path):
    convert_args_ct = {"input" : path_to_dicom_ct_folder,
                       "output-img" : ct_nrrd_path}

    # clean old log file if it exist
    if os.path.exists(log_file_path): os.remove(log_file_path)
    
    pypla.convert(verbose = verbose,
                  path_to_log_file = log_file_path,
                  **convert_args_ct)
    
  # Perform resampling of the original CT nrrd file and save as _CT.nrrd 
  # The header of this is used to create the individual nrrd segments. 
  ct_nrrd_path_resample = os.path.join(pat_dir_nrrd_path, pat_id + "_CT.nrrd") 
  log_file_path = os.path.join(pat_dir_nrrd_path, pat_id + '_pypla_resample.log')
  resample_args_ct_nrrd = {"input" : ct_nrrd_path,
                           "output" : ct_nrrd_path_resample,
                           "spacing": "1 1 2.5"}
  pypla.resample(verbose = verbose,
                path_to_log_file = log_file_path,
                **resample_args_ct_nrrd)


In [ ]:
def resample_softmax_nrrd_files(processed_nrrd_path_nnunet, series_id):

  """Resample the softmax nrrd files from the output of nnUnet back to the
     original space 

  Inputs: 
    processed_nrrd_path_nnunet : the path of the nrrd files from output of nnUNet
    series_id                  : series_id being processed 

  Outputs:
    resamples each of the files from the output of nnUNet back to the original space 

  """
    
  input_folder_path = os.path.join(processed_nrrd_path_nnunet,series_id,"pred_softmax")
  output_folder_path = os.path.join(processed_nrrd_path_nnunet,series_id,"pred_softmax_resample")

  if not os.path.isdir(output_folder_path):
    os.mkdir(output_folder_path)

  nrrd_files_input = [os.path.join(input_folder_path,f) for f in os.listdir(input_folder_path)]
  nrrd_files_output = [os.path.join(output_folder_path,f) for f in os.listdir(input_folder_path)]

  # convert each file 
  for input_file, output_file in zip(nrrd_files_input, nrrd_files_output):
    convert_args_ct = {"input" : input_file,
                      "output" : output_file,
                      "fixed": ct_nifti_path,
                      "interpolation": "linear"}
    resample_log_file = os.path.join(output_folder_path, 'resample.log')
    pypla.resample(verbose = True,
                   path_to_log_file = resample_log_file,
                   **convert_args_ct)
    
    return 

In [ ]:
def nrrd_to_dicomseg_resampled(sorted_base_path, processed_base_path,
                     dicomseg_json_path, pat_id, skip_empty_slices = True):

  """
  Export DICOM SEG object from segmentation masks stored in NRRD files.
  Takes as input the resampled segthor_nrrd file, which is resampled to the 
  original image spacing. 

  Arguments:
    sorted_base_path    : required - path to the folder where the sorted data should be stored.
    processed_base_path : required - path to the folder where the preprocessed NRRD data are stored
    dicomseg_json_path  : required - ...
    pat_id              : required - patient ID (used for naming purposes). 

  Outputs:
    This function [...]
  """

  path_to_ct_dir = os.path.join(sorted_base_path, pat_id, "CT")

  # processed_dicomseg_path = os.path.join(processed_base_path, "dicomseg")
  processed_dicomseg_path = os.path.join(processed_base_path, "nnunet", "dicomseg")
  pat_dir_dicomseg_path = os.path.join(processed_dicomseg_path, pat_id)

  if not os.path.exists(pat_dir_dicomseg_path):
    os.mkdir(pat_dir_dicomseg_path)

  # DK added for now
  # same as processed_nrrd_path_nnunet
  processed_nrrd_path = os.path.join(processed_base_path, "nnunet", "nrrd")

  pred_segmasks_nrrd = os.path.join(processed_nrrd_path, pat_id, pat_id + "_pred_segthor_resampled.nrrd") # instead of _pred_segthor.nrrd 

  dicom_seg_out_path = os.path.join(pat_dir_dicomseg_path, pat_id + "_SEG.dcm")

  # transform from bool to int according to `itkimage2segimage` requirements
  skip_flag = "--skip" if skip_empty_slices == True else ""

  !itkimage2segimage --inputImageList $pred_segmasks_nrrd \
                     --inputDICOMDirectory $path_to_ct_dir \
                     --outputDICOM $dicom_seg_out_path \
                     --inputMetadata $dicomseg_json_path $skip_flag

In [ ]:
def prep_input_data_resample(processed_nifti_path, model_input_folder, pat_id, verbose = True):
  
  """
  After converting the CT to nifti, this function resamples the CT volume and 
  copies the resampled one to the model_input_folder. 

  Arguments:
    processed_nifti_path : path where the converted nifti file is stored 
    model_input_folder   : input folder for nnuNet 
    pat_id               : required - patient ID (used for naming purposes).
  
  Outputs:
    This function [...]
  """

  # FIXME: ok for a notebook; for scripting, change this to `shutil`

  pat_dir_nifti_path = os.path.join(processed_nifti_path, pat_id)
  ct_nifti_path = os.path.join(pat_dir_nifti_path, pat_id + "_CT.nii.gz")

  # Resample, and then copy 
  ct_nifti_path_resample = os.path.join(pat_dir_nifti_path, pat_id + "_CT_resample.nii.gz")
  log_file_path = os.path.join(pat_dir_nifti_path, 'resample_log.log')
  convert_args_ct = {"input" : ct_nifti_path,
                      "output" : ct_nifti_path_resample,
                      "spacing": "1 1 2.5"}
  pypla.resample(verbose = verbose,
                path_to_log_file = log_file_path,
                **convert_args_ct)

  copy_to_path = os.path.join(model_input_folder, pat_id + "_0000.nii.gz")
    
  # copy NIfTI to the right dir for nnU-Net processing
  # delete the resampled file 
  if not os.path.exists(copy_to_path):
    print("Copying %s\nto %s..."%(ct_nifti_path_resample, copy_to_path))
    !cp $ct_nifti_path_resample $copy_to_path
    # !rm $ct_nifti_path_resample 
    print("... Done.")


In [ ]:
def nrrd_to_dicomseg_resampled(sorted_base_path, processed_base_path,
                     dicomseg_json_path, pat_id, skip_empty_slices = True):

  """
  Export DICOM SEG object from segmentation masks stored in NRRD files.
  Takes as input the resampled segthor_nrrd file, which is resampled to the 
  original image spacing. 

  Arguments:
    sorted_base_path    : required - path to the folder where the sorted data should be stored.
    processed_base_path : required - path to the folder where the preprocessed NRRD data are stored
    dicomseg_json_path  : required - ...
    pat_id              : required - patient ID (used for naming purposes). 

  Outputs:
    This function [...]
  """

  path_to_ct_dir = os.path.join(sorted_base_path, pat_id, "CT")

  # processed_dicomseg_path = os.path.join(processed_base_path, "dicomseg")
  processed_dicomseg_path = os.path.join(processed_base_path, "nnunet", "dicomseg")
  pat_dir_dicomseg_path = os.path.join(processed_dicomseg_path, pat_id)

  if not os.path.exists(pat_dir_dicomseg_path):
    os.mkdir(pat_dir_dicomseg_path)

  # DK added for now
  # same as processed_nrrd_path_nnunet
  processed_nrrd_path = os.path.join(processed_base_path, "nnunet", "nrrd")

  pred_segmasks_nrrd = os.path.join(processed_nrrd_path, pat_id, pat_id + "_pred_segthor_resampled.nrrd") # instead of _pred_segthor.nrrd 

  dicom_seg_out_path = os.path.join(pat_dir_dicomseg_path, pat_id + "_SEG.dcm")

  # transform from bool to int according to `itkimage2segimage` requirements
  skip_flag = "--skip" if skip_empty_slices == True else ""

  !itkimage2segimage --inputImageList $pred_segmasks_nrrd \
                     --inputDICOMDirectory $path_to_ct_dir \
                     --outputDICOM $dicom_seg_out_path \
                     --inputMetadata $dicomseg_json_path $skip_flag

In [ ]:
# Run over each series, do the checking if processed in loop
for idx, series_id in enumerate(series_id_list):

  # Run over each model 
  for model_idx, nnunet_model in enumerate(model_list):


    # -----------------
    # nnU-Net pipeline parameters

    experiment_folder_name = nnunet_model + "-tta" if use_tta == True else nnunet_model + "-no_tta"
    bucket_experiment_folder_uri_nnunet = os.path.join(bucket_output_base_uri_nnunet, experiment_folder_name)

    bucket_log_folder_uri_nnunet = os.path.join(bucket_experiment_folder_uri_nnunet, 'log')

    bucket_nifti_folder_uri_nnunet = os.path.join(bucket_experiment_folder_uri_nnunet, 'nii')
    bucket_softmax_pred_folder_uri_nnunet = os.path.join(bucket_experiment_folder_uri_nnunet, 'softmax_pred')

    bucket_dicomseg_folder_uri_nnunet = os.path.join(bucket_experiment_folder_uri_nnunet, 'dicomseg')

    # -----------------
    # save run information

    yaml_fn = "run_params.yaml"
    yaml_out_path = os.path.join(data_base_path, yaml_fn)

    settings_dict = dict()
    settings_dict["bucket"] = dict()
    settings_dict["bucket"]["name"] = bucket_name
    settings_dict["bucket"]["base_uri"] = bucket_base_uri
    settings_dict["bucket"]["output_base_uri"] = bucket_output_base_uri_nnunet
    settings_dict["bucket"]["experiment_folder_uri"] = bucket_experiment_folder_uri_nnunet
    settings_dict["bucket"]["nifti_folder_uri"] = bucket_nifti_folder_uri_nnunet
    settings_dict["bucket"]["softmax_pred_folder_uri"] = bucket_softmax_pred_folder_uri_nnunet
    settings_dict["bucket"]["dicomseg_folder_uri"] = bucket_dicomseg_folder_uri_nnunet
    settings_dict["bucket"]["log_folder_uri"] = bucket_log_folder_uri_nnunet

    settings_dict["inference"] = dict()
    settings_dict["inference"]["model"] = nnunet_model
    settings_dict["inference"]["use_tta"] = use_tta
    settings_dict["inference"]["export_prob_maps"] = export_prob_maps

    with open(yaml_out_path, 'w') as fp:
      yaml.dump(settings_dict, fp, default_flow_style = False)

    gs_uri_yaml_file = os.path.join(bucket_log_folder_uri_nnunet, yaml_fn)

    #!gsutil -m cp $yaml_out_path $gs_uri_yaml_file
    # !/content/s5cmd --endpoint-url https://storage.googleapis.com cp $yaml_out_path $gs_uri_yaml_file
    !s5cmd --endpoint-url https://storage.googleapis.com cp $yaml_out_path $gs_uri_yaml_file


    #  -----------------
    # init

    start_total_nnunet = time.time()

    # init every single time, as the most recent logs are loaded from the bucket
    inference_time_dict_nnunet = dict()
    total_time_dict_nnunet = dict()
    inference_time_dict_bpr = dict()
    total_time_dict_bpr = dict()

    # clear_output(wait = True)

    # print("(%g/%g) Processing series %s"%(idx + 1, len(series_to_process_id_list), series_id))
    print("Processing series %s"%(series_id))

    series_df = cohort_df[cohort_df["SeriesInstanceUID"] == series_id]
    num_instances = series_df['num_instances'].to_list()[0]

    has_segmask_already = False

    dicomseg_fn = series_id + "_SEG.dcm"

    input_nifti_fn = series_id + "_0000.nii.gz"
    input_nifti_path = os.path.join(model_input_folder_nnunet, input_nifti_fn)

    pred_nifti_fn = series_id + ".nii.gz"
    pred_nifti_path = os.path.join(model_output_folder_nnunet, pred_nifti_fn)

    pred_softmax_folder_name = "pred_softmax"
    pred_softmax_folder_path = os.path.join(processed_nrrd_path_nnunet, series_id, pred_softmax_folder_name)

    # -----------------
    # GS URI definition

    # gs URI at which the *nii.gz object is or will be stored in the bucket
    gs_uri_nifti_file = os.path.join(bucket_nifti_folder_uri_nnunet, pred_nifti_fn)

    # gs URI at which the folder storing the *.nrrd softmax probabilities is or will be stored in the bucket
    gs_uri_softmax_pred_folder = os.path.join(bucket_softmax_pred_folder_uri_nnunet, series_id)

    # gs URI at which the DICOM SEG object is or will be stored in the bucket
    gs_uri_dicomseg_file = os.path.join(bucket_dicomseg_folder_uri_nnunet, dicomseg_fn)

    # DK added - gs URI at which the CT to nii file is or will be stored in the bucket 
    gs_uri_ct_nifti_file = os.path.join(bucket_dicomseg_folder_uri_nnunet, pred_nifti_fn)


    # -----------------
    # cross-load the CT data from the IDC buckets, run the preprocessing

    # check whether the NIfTI seg mask exists already
    has_segmask_already = file_exists_in_bucket(project_name = project_name,
                                                bucket_name = bucket_name,
                                                file_gs_uri = gs_uri_nifti_file)

    # Query to see if the series for the specific model has been run 
    field_name = 'inference_time_' + experiment_folder_name.replace('-', '_')
    value_found = check_value_exists_in_table(nnunet_table_id_fullname, project_name, series_id, field_name)
    print ('value_found inference_time nnunet: ' + str(value_found))

    # if the seg mask doesn't exist, and the value doesn't exist in the table, do the processing 
    # if has_segmask_already == False or value_found == 0: 
    # if True: # for now 
    if has_segmask_already == False: 

      # if the raw segmentation file exists in the output directory but the DICOM SEG
      # doesn't, skip the inference phase. Data still need to be downloaded because
      # the DICOM folder is essential in the DICOM SEG generation process
      # If the download_path already exists, we don't need to download it again for this series 
      
      # download_path = os.path.join(raw_base_path, series_id)
      # if not os.path.exists(download_path):
      download_path = os.path.join(sorted_base_path, series_id) # should only be deleted after all models are run for the series, not after bpr like before.
      if not os.path.exists(download_path):
      # if True: # for now
        start_time_download_series_data = time.time()
        download_series_data_s5cmd(raw_base_path = raw_base_path,
                                    sorted_base_path = sorted_base_path,
                                    series_df = series_df,
                                    remove_raw = True)
        elapsed_time_download_series_data = time.time()-start_time_download_series_data

        # DICOM CT to NRRD - good to have for a number of reasons
        # pypla_dicom_ct_to_nrrd(sorted_base_path = sorted_base_path,
        #                         processed_nrrd_path = processed_nrrd_path_nnunet,
        #                         pat_id = series_id, 
        #                         verbose = True)
        pypla_dicom_ct_to_nrrd_resample(sorted_base_path = sorted_base_path,
                                        processed_nrrd_path = processed_nrrd_path_nnunet,
                                        pat_id = series_id, 
                                        verbose = True)

        # DICOM CT to NIfTI - required for the processing
        start_time_ct_to_nii = time.time()
        pypla_dicom_ct_to_nifti(sorted_base_path = sorted_base_path,
                                processed_nifti_path = processed_nifti_path,
                                pat_id = series_id, 
                                verbose = True)
        elapsed_time_ct_to_nii = time.time()-start_time_ct_to_nii 

        # added DK
        # upload to bucket 
        ct_nifti_path = os.path.join(processed_nifti_path,series_id,series_id+"_CT.nii.gz")
        # !gsutil -m cp $ct_nifti_path $gs_uri_ct_nifti_file 
        # !/content/s5cmd --endpoint-url https://storage.googleapis.com cp $ct_nifti_path $gs_uri_ct_nifti_file
        !s5cmd --endpoint-url https://storage.googleapis.com cp $ct_nifti_path $gs_uri_ct_nifti_file


        # # prepare the `model_input` folder for the inference phase
        # prep_input_data(processed_nifti_path = processed_nifti_path,
        #                 model_input_folder = model_input_folder_nnunet,
        #                 pat_id = series_id)
        # prepare the `model_input` folder for the inference phase
        # resample the ct image to 1x1x2.5 before using in inference 
        prep_input_data_resample(processed_nifti_path = processed_nifti_path,
                                  model_input_folder = model_input_folder_nnunet,
                                  pat_id = series_id, 
                                  verbose = True)



      start_inference_nnunet = time.time()
      # run the DL-based prediction
      process_patient_nnunet(model_input_folder = model_input_folder_nnunet,
                            model_output_folder = model_output_folder_nnunet, 
                            nnunet_model = nnunet_model, use_tta = use_tta,
                            export_prob_maps = export_prob_maps, verbose = True)

      elapsed_inference_nnunet = time.time() - start_inference_nnunet
      # inference_time_dict_nnunet[series_id] = elapsed_inference_nnunet

      # Need to create a CT nrrd file that is resampled to 1x1x2.5 -- the header of this 
      # is used as input to create the individual nrrd segments. 
      # The nrrd in /content/data/processed/nlst/nnunet/nrrd/1.3.6.1.4.1.14519.5.2.1.7009.9004.120988899216969247850007613978/1.3.6.1.4.1.14519.5.2.1.7009.9004.120988899216969247850007613978_CT.nrrd
      # is in the original space. 

      # get the dim from the original file 
      ct_nifti_path = os.path.join(processed_nifti_path,series_id,series_id+"_CT.nii.gz")
      nii = nib.load(ct_nifti_path)
      spacing = [nii.header['pixdim'][1], nii.header['pixdim'][2], nii.header['pixdim'][3]]
      # spacing_str = " ".join([spacing[0], spacing[1], spacing[2]])
      spacing_str = " ".join([str(s) for s in spacing])

      if export_prob_maps: 
        # convert the softmax predictions to NRRD files
        numpy_to_nrrd(model_output_folder = model_output_folder_nnunet,
                      processed_nrrd_path = processed_nrrd_path_nnunet,
                      pat_id = series_id,
                      output_folder_name = pred_softmax_folder_name)
        
        ### Resample the nrrd files back to the original space before copying to the bucket ### 
        # This should be linear interpolation because we are resampling the softmax predictions.
        # get list of files that need to be converted 
        input_folder_path = os.path.join(processed_nrrd_path_nnunet,series_id,"pred_softmax")
        output_folder_path = os.path.join(processed_nrrd_path_nnunet,series_id,"pred_softmax_resample")
        if not os.path.isdir(output_folder_path):
          os.mkdir(output_folder_path)
        nrrd_files_input = [os.path.join(input_folder_path,f) for f in os.listdir(input_folder_path)]
        nrrd_files_output = [os.path.join(output_folder_path,f) for f in os.listdir(input_folder_path)]
        # convert each file 
        for input_file, output_file in zip(nrrd_files_input, nrrd_files_output):
          # convert_args_ct = {"input" : input_file,
          #                     "output" : output_file,
          #                     "spacing": spacing_str,
          #                     "interpolation": "linear"}
          convert_args_ct = {"input" : input_file,
                            "output" : output_file,
                            "fixed": ct_nifti_path,
                            "interpolation": "linear"}
          resample_log_file = os.path.join(output_folder_path, 'resample.log')
          pypla.resample(verbose = True,
                        path_to_log_file = resample_log_file,
                        **convert_args_ct)

      # copy the nnU-Net *.npz softmax probabilities in the chosen bucket
      # !gsutil -m cp $pred_softmax_folder_path/* $gs_uri_softmax_pred_folder
      if (export_prob_maps):
        # !/content/s5cmd --endpoint-url https://storage.googleapis.com cp $pred_softmax_folder_path/ $gs_uri_softmax_pred_folder/
        # !/content/s5cmd --endpoint-url https://storage.googleapis.com cp $output_folder_path/ $gs_uri_softmax_pred_folder/
        !s5cmd --endpoint-url https://storage.googleapis.com cp $output_folder_path/ $gs_uri_softmax_pred_folder/

      

      ### Resample the nii.gz file to original space before copying to bucket 
      # copy the nnU-Net *.nii.gz binary masks in the chosen bucket
      # !gsutil -m cp $pred_nifti_path $gs_uri_nifti_file
      # !/content/s5cmd --endpoint-url https://storage.googleapis.com cp $pred_nifti_path $gs_uri_nifti_file
      pred_nifti_path_resample = os.path.join(model_output_folder_nnunet, series_id + '_resample.nii.gz')
      # convert_args_ct = {"input" : pred_nifti_path,
      #                     "output" : pred_nifti_path_resample,
      #                     "spacing": spacing_str, 
      #                     "interpolation": "nn"}
      convert_args_ct = {"input" : pred_nifti_path,
                          "output" : pred_nifti_path_resample,
                          "fixed": ct_nifti_path, 
                          "interpolation": "nn"}
      pred_nifti_path_resample_log_file = os.path.join(model_output_folder_nnunet, 'resample.log')
      pypla.resample(verbose = True,
                    path_to_log_file = pred_nifti_path_resample_log_file,
                    **convert_args_ct)
      # !/content/s5cmd --endpoint-url https://storage.googleapis.com cp $pred_nifti_path_resample $gs_uri_nifti_file
      !s5cmd --endpoint-url https://storage.googleapis.com cp $pred_nifti_path_resample $gs_uri_nifti_file

      
      
      # remove the NIfTI file the prediction was computed from
      # !rm $input_nifti_path

        
      # -----------------
      # post-processing
      pypla_postprocess(processed_nrrd_path = processed_nrrd_path_nnunet,
                        model_output_folder = model_output_folder_nnunet,
                        pat_id = series_id)
      
      ### Resample the nrrd file back to the original space, before converting to dicomseg ### 
      pred_nrrd_path_input = os.path.join(processed_nrrd_path_nnunet, series_id, series_id + "_pred_segthor.nrrd")
      pred_nrrd_path_output = os.path.join(processed_nrrd_path_nnunet, series_id, series_id + "_pred_segthor_resampled.nrrd")
      resample_log_file = os.path.join(processed_nrrd_path_nnunet, series_id, series_id + '_resample_log.log')
      # convert_args_ct = {"input" : pred_nrrd_path_input,
      #                     "output" : pred_nrrd_path_output,
      #                     "spacing": spacing_str, 
      #                     "interpolation": "nn"}
      convert_args_ct = {"input" : pred_nrrd_path_input,
                        "output" : pred_nrrd_path_output,
                        "fixed": ct_nifti_path, 
                        "interpolation": "nn"}
      pypla.resample(verbose = True,
              path_to_log_file = resample_log_file,
              **convert_args_ct)

      
      # Modify the dicomseg_json file so that the SegmentAlgorithmName is representative of the model and other parameters 
      # Writes out the json file 
      SegmentAlgorithmName = experiment_folder_name 
      dicomseg_json_path_modified = "/content/data/dicomseg_metadata_" + SegmentAlgorithmName + '.json'
      modify_dicomseg_json_file(dicomseg_json_path, dicomseg_json_path_modified, SegmentAlgorithmName)
      # upload the json file 
      gs_uri_dicomseg_json_file = os.path.join(bucket_experiment_folder_uri_nnunet, 'dicomseg_metadata_' + SegmentAlgorithmName + '.json')
      # !/content/s5cmd --endpoint-url https://storage.googleapis.com cp $dicomseg_json_path_modified $gs_uri_dicomseg_json_file
      !s5cmd --endpoint-url https://storage.googleapis.com cp $dicomseg_json_path_modified $gs_uri_dicomseg_json_file


      # nrrd_to_dicomseg(sorted_base_path = sorted_base_path,
      #                  processed_base_path = processed_base_path,
      #                  dicomseg_json_path = dicomseg_json_path_modified, # dicomseg_json_path
      #                  pat_id = series_id)
      nrrd_to_dicomseg_resampled(sorted_base_path = sorted_base_path,
                  processed_base_path = processed_base_path,
                  dicomseg_json_path = dicomseg_json_path_modified, # dicomseg_json_path
                  pat_id = series_id)

      pred_dicomseg_path = os.path.join(processed_dicomseg_path_nnunet, series_id, dicomseg_fn)

      # !gsutil -m cp $pred_dicomseg_path $gs_uri_dicomseg_file
      # !/content/s5cmd --endpoint-url https://storage.googleapis.com cp $pred_dicomseg_path $gs_uri_dicomseg_file
      !s5cmd --endpoint-url https://storage.googleapis.com cp $pred_dicomseg_path $gs_uri_dicomseg_file


      # delete the files in the pat_dir_dicomseg_path
      processed_dicomseg_path = os.path.join(processed_base_path, "nnunet", "dicomseg")
      pat_dir_dicomseg_path = os.path.join(processed_dicomseg_path, series_id)
      print ('removing files from: ' + pat_dir_dicomseg_path)
      !rm -r $pat_dir_dicomseg_path/*
      !rmdir $pat_dir_dicomseg_path

      # remove files from the nnunet output prediction 
      !rm /content/data/nnunet/nnunet_output/*.nii.gz
      !rm /content/data/nnunet/nnunet_output/*.npz
      !rm /content/data/nnunet/nnunet_output/*.pkl
      !rm /content/data/nnunet/nnunet_output/*.json

      # # delete the files in the processed_nrrd_path_nnunet
      # # but don't want to delete the _CT.nrrd file, the rest are the segments
      # print ('removing files excluding CT.nrrd from: ' + processed_nrrd_path_nnunet)
      # # !rm -r $processed_nrrd_path_nnunet/*
      processed_nrrd_path_nnunet_segments = os.path.join(processed_nrrd_path_nnunet,series_id,'pred_softmax')
      !rm $processed_nrrd_path_nnunet_segments/*.nrrd
      !rm $processed_nrrd_path_nnunet/$series_id/*.log
      !rm $processed_nrrd_path_nnunet/$series_id/*_pred_segthor*.nrrd
      # delete the files in the resampled directory 
      processed_nrrd_path_nnunet_segments_resampled = os.path.join(processed_nrrd_path_nnunet,series_id,'pred_softmax_resampled')
      !rm $processed_nrrd_path_nnunet_segments_resampled/*.nrrd

      elapsed_total_nnunet = time.time() - start_total_nnunet

      print("End-to-end processing of %s completed in %g seconds.\n"%(series_id, elapsed_total_nnunet))

      ### Inference/total time ### 
      # dict_to_add = dict()
      # dict_to_add[series_id] = [num_instances, elapsed_inference_nnunet, elapsed_total_nnunet]
      # save_time_and_num_instances_to_table(project_name, dataset_id, multiple_models_time_table_id_name_nnunet, dict_to_add)
      ### Using INSERT ### 
      column = nnunet_model + "-tta" if use_tta == True else nnunet_model + "-no_tta"
      column = column.replace('-', '_')
      column_inference = 'inference_time_' + column
      column_total = 'total_time_' + column
      SeriesInstanceUID_column = "SeriesInstanceUID"
      num_instances_column = "num_instances"
      # client = bigquery.Client(project=project_name)
      client = bigquery.Client()
      query = f"""
        INSERT INTO
          {nnunet_table_id_fullname}({SeriesInstanceUID_column}, 
                                    {num_instances_column}, 
                                    {column_inference},
                                    {column_total})
        VALUES
          (@SeriesInstanceUID_value, @num_instances_value, @inference_time_value, @total_time_value)
          """
      job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ScalarQueryParameter("SeriesInstanceUID_value", "STRING", series_id), 
                                                            bigquery.ScalarQueryParameter("num_instances_value", "INTEGER", int(num_instances)), 
                                                            bigquery.ScalarQueryParameter("inference_time_value", "FLOAT", elapsed_inference_nnunet), 
                                                            bigquery.ScalarQueryParameter("total_time_value", "FLOAT", elapsed_total_nnunet)])
      result = client.query(query, job_config=job_config)


    ### only run BPR once ### 

    if (model_idx==0):

      # -----------------
      # BPR 

      start_total_bpr = time.time()

      input_nifti_fn = series_id + ".nii.gz"
      input_nifti_path = os.path.join(model_input_folder_bpr, input_nifti_fn)

      pred_json_fn = series_id + ".json"
      pred_json_path = os.path.join(model_output_folder_bpr, pred_json_fn)

      # gs URI at which the *.json object is or will be stored in the bucket
      gs_uri_json_file = os.path.join(bucket_json_folder_uri_bpr, pred_json_fn)
      has_json_already = file_exists_in_bucket(project_name = project_name,
                                              bucket_name = bucket_name,
                                              file_gs_uri = gs_uri_json_file)

      # Query to see if the series for the specific model has been run 
      field_name = 'inference_time_bpr' 
      value_found = check_value_exists_in_table(bpr_table_id_fullname, project_name, series_id, field_name)
      print ('value_found inference_time_bpr: ' + str(value_found))

      # -----------------
      # DL-inference for BPR 
      # json not found and timing entry doesn't exist in table 
      # if not has_json_already or value_found == 0:
      if has_json_already == False: 

        # If ct dicom directory does not exist, download the ct files 
        ct_dicom_path = os.path.join(sorted_base_path, series_id) 
        if not os.path.isdir(ct_dicom_path):
          start_time_download_series_data = time.time()
          download_series_data_s5cmd(raw_base_path = raw_base_path,
                                    sorted_base_path = sorted_base_path,
                                    series_df = series_df,
                                    remove_raw = True)
          elapsed_time_download_series_data = time.time()-start_time_download_series_data

        # If ct nifti doesn't exist, create it 
        pat_dir_nifti_path = os.path.join(processed_nifti_path, series_id)
        ct_nifti_path = os.path.join(pat_dir_nifti_path, series_id + "_CT.nii.gz")
        if not os.path.exists(ct_nifti_path):
          start_time_ct_to_nii = time.time()
          pypla_dicom_ct_to_nifti(sorted_base_path = sorted_base_path,
                                  processed_nifti_path = processed_nifti_path,
                                  pat_id = series_id, 
                                  verbose = True)
          elapsed_time_ct_to_nii = time.time()-start_time_ct_to_nii 

        # prepare the `model_input` folder for the inference phase
        prep_input_data_bpr(processed_nifti_path = processed_nifti_path,
                            model_input_folder = model_input_folder_bpr,
                            pat_id = series_id)

        # single patient needs a temporary folder, as bpr inference takes as input
        # a directory

        model_input_folder_series = os.path.join(model_input_folder_bpr,series_id+'.nii.gz')
        !cp $model_input_folder_series $model_input_folder_tmp_bpr

        start_inference_bpr = time.time()
        process_patient_bpr(model_input_folder = model_input_folder_tmp_bpr,
                            model_output_folder = model_output_folder_tmp_bpr,
                            model = "/content/models/bpr_model/public_bpr_model/")
        elapsed_inference_bpr = time.time() - start_inference_bpr
        inference_time_dict_bpr[series_id] = elapsed_inference_bpr

        # !cp $model_output_folder_tmp/* $model_output_folder
        !cp $model_output_folder_tmp_bpr/* $model_output_folder_bpr

        # copy the bpr *.json prediction in the chosen bucket 
        s_time = time.time()
        # !gsutil -m cp $pred_json_path $gs_uri_json_file 
        # !/content/s5cmd --endpoint-url https://storage.googleapis.com cp $pred_json_path $gs_uri_json_file 
        !s5cmd --endpoint-url https://storage.googleapis.com cp $pred_json_path $gs_uri_json_file 

        e_time = time.time()
        print ('Copy json file to bucket: ' + str(e_time-s_time))

        # -----------------
        # remove files from BPR 

        # remove the file from the model_input_tmp folder
        model_input_folder_tmp_series = os.path.join(model_input_folder_tmp_bpr,series_id+'.nii.gz')
        !rm $model_input_folder_tmp_series

        # remove the file from the model_output_tmp folder 
        model_output_folder_tmp_series = os.path.join(model_output_folder_tmp_bpr,series_id+'.json')
        !rm $model_output_folder_tmp_series

        # remove the NIfTI file the prediction was computed from
        !rm $input_nifti_path

        # Remove the raw patient data 
        # sorted_base_path_remove = os.path.join(sorted_base_path,series_id)
        # !rm -r $sorted_base_path_remove 

        # Delete json from bpr_output folder 
        model_output_folder_series = os.path.join(model_output_folder_bpr,series_id+'.json')
        !rm $model_output_folder_series

        # Delete the files in /content/data/processed/bpr/nii
        # !rm -r $processed_nifti_path/* ## this was for nnUNet 
        !rm -r /content/data/processed/bpr/nii
        
        # should be in the if statement for if it has a json file. 
        # because only then do you want to add the value to the table.
        elapsed_total_bpr = time.time() - start_total_bpr
        total_time_dict_bpr[series_id] = elapsed_total_bpr
        print("End-to-end processing of %s completed in %g seconds.\n"%(series_id, elapsed_total_bpr))

        # -----------------
        # save inference time/total time information 

        # elapsed_total_bpr = (time.time() - start_total_bpr) + elapsed_time_download_series_data + elapsed_time_ct_to_nii
        

        # ### Inference time ### 
        # dict_to_add = dict()
        # dict_to_add[series_id] = [elapsed_inference_bpr, num_instances]
        # save_time_and_num_instances_to_table(project_name, dataset_id, inference_time_table_id_name_bpr, dict_to_add)
        # ### Total time ###
        # dict_to_add = dict()
        # dict_to_add[series_id] = [elapsed_total_bpr, num_instances]
        # save_time_and_num_instances_to_table(project_name, dataset_id, total_time_table_id_name_bpr, dict_to_add)

        # dict_to_add = dict()
        # dict_to_add[series_id] = [num_instances, elapsed_inference_bpr, elapsed_total_bpr]
        # add_to_df = pd.DataFrame.from_dict(data = dict_to_add, orient = "index")
        # add_to_df = add_to_df.reset_index()
        # add_to_df = add_to_df.rename(columns = {"index" : 'SeriesInstanceUID', 
        #                                         0 : 'num_instances', 
        #                                         1 : 'inference_time_bpr', 
        #                                         2 : 'total_time_bpr'})
        # client = bigquery.Client(project=project_name)
        # job_config = bigquery.LoadJobConfig()
        # job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
        # job = client.load_table_from_dataframe(add_to_df, bpr_table_id_fullname, job_config=job_config)

        ### Using INSERT ### 
        column_inference = 'inference_time_bpr'
        column_total = 'total_time_bpr'
        SeriesInstanceUID_column = "SeriesInstanceUID"
        num_instances_column = "num_instances"
        # client = bigquery.Client(project=project_name)
        client = bigquery.Client()
        query = f"""
          INSERT INTO
            {bpr_table_id_fullname}({SeriesInstanceUID_column}, 
                                    {num_instances_column}, 
                                    {column_inference},
                                    {column_total})
          VALUES
            (@SeriesInstanceUID_value, @num_instances_value, @inference_time_value, @total_time_value)
            """
        job_config = bigquery.QueryJobConfig(query_parameters=[bigquery.ScalarQueryParameter("SeriesInstanceUID_value", "STRING", series_id), 
                                                              bigquery.ScalarQueryParameter("num_instances_value", "INTEGER", int(num_instances)), 
                                                              bigquery.ScalarQueryParameter("inference_time_value", "FLOAT", elapsed_inference_bpr), 
                                                              bigquery.ScalarQueryParameter("total_time_value", "FLOAT", elapsed_total_bpr)])
        result = client.query(query, job_config=job_config)


  # Remove some files/folders to get ready for the next series run
  print ('*****deleting files for the next series run ******')
  if os.listdir('/content/data/raw/nlst/dicom'):
    !rm -r /content/data/raw/nlst/dicom/*  
  if os.listdir('/content/data/processed/nlst/nii'):
    !rm -r /content/data/processed/nlst/nii/*
  # remove from model input 
  !rm /content/data/nnunet/model_input/*

  # Don't need to remove this -- alreadty removed above by '/content/data/raw/nlst/dicom
  # Remove the raw series data to get ready for the next series run 
  # sorted_base_path_remove = os.path.join(sorted_base_path,series_id)
  # !rm -r $sorted_base_path_remove 

  # Remove the series data from /content/data/processed/nlst/nnunet/nrrd/
  !rm -r /content/data/processed/nlst/nnunet/nrrd/*

Streaming output truncated to the last 5000 lines.
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source image item
Added source 

# DO NOT RUN. WIP queries

First let's get the list of SeriesInstanceUIDs for the subset we want to investigate.

In [ ]:
### Get the list of series_ids ### 

# First query to get the list of series uids 
table_view_id_name_full = '.'.join([project_name, dataset_table_id, table_view_id_name])
if (gce_vm):
  client = bigquery.Client()
else: 
  client = bigquery.Client(project=project_name)

query = f"""
  SELECT 
    DISTINCT(SeriesInstanceUID)
  FROM
    {table_view_id_name_full}
  """
  # LIMIT 50 OFFSET @patient_offset; -- need the same order though.

start_time = time.time()
# job_config = bigquery.QueryJobConfig(query_parameters=[
#                                                        bigquery.ScalarQueryParameter("patient_offset", "INTEGER", int(patient_offset))
#                                                        ])
job_config = bigquery.QueryJobConfig()
result = client.query(query, job_config=job_config) 
series_df = result.to_dataframe(create_bqstorage_client=True)
end_time = time.time()
print ('elapsed time: ' + str(end_time-start_time)) # 2.7 seconds. 
series_id_list_temp = sorted(list(series_df['SeriesInstanceUID'].values))
series_id_list = series_id_list_temp[patient_offset:patient_offset+50]
series_id_list = [str(f) for f in series_id_list]

print ('num of unique series should be 50 -- the number of unique series is: ' + str(len(set(series_id_list))))

elapsed time: 2.4714624881744385
num of unique series should be 50 -- the number of unique series is: 50


In [ ]:
# series_id_strip_list = [f.strip() for f in series_id_list]
# print(series_id_list)
# print(len(series_id_list))
# print(len(set(series_id_list)))
asd = [True if a == b else False for a,b in zip(series_id_list, [f.strip() for f in series_id_list])]
print(asd)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [ ]:
series_id_list_bq = pd.read_csv('series_id_list_bq.csv')
series_id_list_bq_list = series_id_list_bq_list = series_id_list_bq['SeriesInstanceUID'].values 

diff = set(series_id_list) - set(series_id_list_bq_list)
print(diff)

{'1.2.840.113654.2.55.139643264962705498172562651504603085682'}


Then let's get the rest of the fields for that set of SeriesInstanceUIDs. 

In [ ]:
# Then get the subset cohort df of those series 

table_view_id_name_full = '.'.join([project_name, dataset_table_id, table_view_id_name])
print(table_view_id_name_full)

if (gce_vm):
  client = bigquery.Client()
else:
  client = bigquery.Client(project=project_name)
  
query = f"""
  SELECT 
    SeriesInstanceUID,
    PatientID,
    StudyInstanceUID,
    Modality, 
    num_instances,
    num_differences,
    max_difference, 
    min_difference,
    idc_url,
    gcs_url 
  FROM
    {table_view_id_name_full}
  WHERE
    SeriesInstanceUID IN UNNEST(@series_id_list);
  """

start_time = time.time()
job_config = bigquery.QueryJobConfig(query_parameters=[
                                                       bigquery.ArrayQueryParameter("series_id_list", "STRING", series_id_list)
                                                       ])
result = client.query(query, job_config=job_config) 
cohort_df = result.to_dataframe(create_bqstorage_client=True)
# cohort_df = result.to_dataframe()
end_time = time.time()
print ('elapsed time: ' + str(end_time-start_time)) #  8.6 seconds.



idc-external-018.dataset_nlst.nlst_pick_single_series_per_study_cancer_positive_filtering_missing_slices_axial
elapsed time: 2.3860726356506348


In [ ]:
# get the distinct gcsurls 
# then query to get matching series ids
# should be 50


table_view_id_name_full = '.'.join([project_name, dataset_table_id, table_view_id_name])
print(table_view_id_name_full)

if (gce_vm):
  client = bigquery.Client()
else:
  client = bigquery.Client(project=project_name)
  
query = f"""
  SELECT 
    gcs_url 
  FROM
    {table_view_id_name_full}
  WHERE
    SeriesInstanceUID IN UNNEST(@series_id_list);
  """

start_time = time.time()
job_config = bigquery.QueryJobConfig(query_parameters=[
                                                       bigquery.ArrayQueryParameter("series_id_list", "STRING", series_id_list)
                                                       ])
result = client.query(query, job_config=job_config) 
gcs_url_df = result.to_dataframe(create_bqstorage_client=True)
end_time = time.time()
print ('elapsed time: ' + str(end_time-start_time)) 


idc-external-018.dataset_nlst.nlst_pick_single_series_per_study_cancer_positive_filtering_missing_slices_axial
elapsed time: 6.547682285308838


In [ ]:
gcs_url_list = gcs_url_df['gcs_url'].values 


<class 'str'>


In [ ]:
table_view_id_name_full = '.'.join([project_name, dataset_table_id, table_view_id_name])
print(table_view_id_name_full)

if (gce_vm):
  client = bigquery.Client()
else:
  client = bigquery.Client(project=project_name)
  
query = f"""
  SELECT 
    SeriesInstanceUID
  FROM
    {table_view_id_name_full}
  WHERE
    gcs_url IN UNNEST(@gcs_url_list);
  """

start_time = time.time()
job_config = bigquery.QueryJobConfig(query_parameters=[
                                                       bigquery.ArrayQueryParameter("gcs_url_list", "STRING", list(gcs_url_list))
                                                       ])
result = client.query(query, job_config=job_config) 
series_df = result.to_dataframe(create_bqstorage_client=True)
end_time = time.time()
print ('elapsed time: ' + str(end_time-start_time)) 

idc-external-018.dataset_nlst.nlst_pick_single_series_per_study_cancer_positive_filtering_missing_slices_axial
elapsed time: 8.08346700668335


In [ ]:
len(set(series_df['SeriesInstanceUID'].values))

40

In [ ]:
print(len(set(list(cohort_df['SeriesInstanceUID'].values))))

print(len(set(series_id_list)))



39
50


# DO NOT RUN - Queries that are causing issues!

If the view does not exist, create it from the query loaded from the query.txt file. If the view does exist and overwrite_view = 1, overwrite the view. If the view does exist and overwrite_view = 0, do not overwrite the view. 

In [ ]:
if (gce_vm):
  client = bigquery.Client()
else: 
  client = bigquery.Client(project=project_name)

view_id = '.'.join([project_name, dataset_table_id, table_view_id_name])
print (view_id)

view_exists = 0

view = bigquery.Table(view_id)
view.view_query = query

try:
  view = client.create_table(view)
  view_exists = 0 
except:
  print ('view ' + str(view) + ' already exists')
  view_exists = 1 

print ('view_exists: ' + str(view_exists))

if (overwrite_view):
  print ('view ' + str(view) + ' exists, overwriting view')
  view = bigquery.Table(view_id)
  view.view_query = query
  view = client.update_table(view, ["view_query"]) 

idc-external-018.dataset_nlst.nlst_table_from_query_txt
view idc-external-018.dataset_nlst.nlst_table_from_query_txt already exists
view_exists: 1
view idc-external-018.dataset_nlst.nlst_table_from_query_txt exists, overwriting view


In [ ]:
# Get the series ids from the view 

# First query to get the list of series uids 
table_view_id_name_full = '.'.join([project_name, dataset_table_id, table_view_id_name])
print(table_view_id_name_full)

if (gce_vm):
  client = bigquery.Client()
else: 
  client = bigquery.Client(project=project_name)

query_view = f"""
  SELECT 
    DISTINCT(SeriesInstanceUID)
  FROM
    {table_view_id_name_full};
  LIMIT 50 OFFSET @patient_offset;
  """
  # LIMIT 50 OFFSET @patient_offset; -- need the same order though.

start_time = time.time()
job_config = bigquery.QueryJobConfig(query_parameters=[
                                                       bigquery.ScalarQueryParameter("patient_offset", "INTEGER", int(patient_offset))
                                                       ])
# job_config = bigquery.QueryJobConfig()
result = client.query(query_view, job_config=job_config) 
series_df = result.to_dataframe(create_bqstorage_client=True)
end_time = time.time()
print ('elapsed time: ' + str(end_time-start_time)) # 2.7 seconds. 

series_id_list_temp = sorted(list(series_df['SeriesInstanceUID'].values))
series_id_list = series_id_list_temp[patient_offset:patient_offset+50]
series_id_list = [str(f) for f in series_id_list]

# series_id_list = list(series_df['SeriesInstanceUID'].values)

print ('num of unique series should be 50 -- the number of unique series is: ' + str(len(set(series_id_list))))

idc-external-018.dataset_nlst.nlst_table_from_query_txt
elapsed time: 6.142829895019531
num of unique series should be 50 -- the number of unique series is: 50


In [ ]:
# Then get the subset cohort df of those series 

table_view_id_name_full = '.'.join([project_name, dataset_table_id, table_view_id_name])
print(table_view_id_name_full)

if (gce_vm):
  client = bigquery.Client()
else:
  client = bigquery.Client(project=project_name)
  
query_view = f"""
  SELECT 
    * 
  FROM
    {table_view_id_name_full}
  WHERE
    SeriesInstanceUID IN UNNEST(@series_id_list);
  """

start_time = time.time()
job_config = bigquery.QueryJobConfig(query_parameters=[
                                                       bigquery.ArrayQueryParameter("series_id_list", "STRING", series_id_list)
                                                       ])
result = client.query(query_view, job_config=job_config) 
cohort_df = result.to_dataframe(create_bqstorage_client=True)
end_time = time.time()
print ('elapsed time: ' + str(end_time-start_time)) #  8.6 seconds.

# SeriesInstanceUID,
# PatientID,
# StudyInstanceUID,
# Modality, 
# num_instances,
# num_differences,
# max_difference, 
# min_difference,
# idc_url,
# gcs_url 

idc-external-018.dataset_nlst.nlst_table_from_query_txt
elapsed time: 2.640249490737915


In [ ]:
len(set(list(cohort_df['SeriesInstanceUID'].values)))

37

# DO NOT RUN - Old way of creating view 

In [ ]:
if (gce_vm):
  client = bigquery.Client()
else: 
  client = bigquery.Client(project=project_name)

view_id = '.'.join([project_name, dataset_table_id, table_view_id_name])
print (view_id)

view_exists = 0

view = bigquery.Table(view_id)
view.view_query = query

try:
  view = client.create_table(view)
  view_exists = 0 
except:
  print ('view ' + str(view) + ' already exists')
  view_exists = 1 

print ('view_exists: ' + str(view_exists))

if (overwrite_view):
  print ('view ' + str(view) + ' exists, overwriting view')
  view = bigquery.Table(view_id)
  view.view_query = query
  view = client.update_table(view, ["view_query"]) 

Run the query from the text file to get the df. 

In [ ]:
if (gce_vm):
  client = bigquery.Client()
else: 
  client = bigquery.Client(project=project_name)

job_config = bigquery.QueryJobConfig()
result = client.query(query, job_config=job_config) 
cohort_df_all = result.to_dataframe(create_bqstorage_client=True)
end_time = time.time()
print ('elapsed time: ' + str(end_time-start_time))

cohort_df_all


elapsed time: 1253.3862745761871


,SeriesInstanceUID,PatientID,StudyInstanceUID,Modality,num_instances,num_differences,max_difference,min_difference,idc_url,gcs_url
0,1.3.6.1.4.1.14519.5.2.1.7009.9004.165521089654...,218013,1.3.6.1.4.1.14519.5.2.1.7009.9004.160416029404...,CT,170,2,1.800000000,1.400000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/fcd8dfc1-07b9-4f4e-ac...
1,1.3.6.1.4.1.14519.5.2.1.7009.9004.180073756539...,209306,1.3.6.1.4.1.14519.5.2.1.7009.9004.580155796605...,CT,313,2,1.000000000,0.900000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/2c2a9f1d-1e39-41b5-af...
2,1.3.6.1.4.1.14519.5.2.1.7009.9004.180073756539...,209306,1.3.6.1.4.1.14519.5.2.1.7009.9004.580155796605...,CT,313,2,1.000000000,0.900000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/c9078935-5ea7-449e-92...
3,1.2.840.113654.2.55.12134616178609872830868013...,109589,1.2.840.113654.2.55.29851631757798774022898277...,CT,103,1,3.000000000,3.000000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/3b437734-da79-4150-a0...
4,1.3.6.1.4.1.14519.5.2.1.7009.9004.180073756539...,209306,1.3.6.1.4.1.14519.5.2.1.7009.9004.580155796605...,CT,313,2,1.000000000,0.900000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/8033c4c1-2bde-4d52-a5...
...,...,...,...,...,...,...,...,...,...,...
169726,1.2.840.113654.2.55.23796069011005848477286383...,122262,1.2.840.113654.2.55.63352920373318227847962644...,CT,156,1,2.500000000,2.500000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/4b5580e6-8a5e-4455-b7...
169727,1.3.6.1.4.1.14519.5.2.1.7009.9004.225506400203...,202611,1.3.6.1.4.1.14519.5.2.1.7009.9004.891299100258...,CT,128,1,2.500000000,2.500000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/aaf75851-9295-4740-82...
169728,1.2.840.113654.2.55.27726266020012662899808074...,118553,1.2.840.113654.2.55.14603833779263581028724050...,CT,114,1,2.500000000,2.500000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/3aa04b11-abd3-48e6-bb...
169729,1.2.840.113654.2.55.14932817028091369182940601...,113103,1.2.840.113654.2.55.80636030874872024723867844...,CT,159,1,2.500000000,2.500000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/0653df58-59ca-4b3d-bd...


Get the list of series_id we want to process 

In [ ]:
series_id_list_temp = sorted(list(set(cohort_df_all['SeriesInstanceUID'].values)))
series_id_list = series_id_list_temp[patient_offset:patient_offset+50]
series_id_list = [str(f) for f in series_id_list]

Form the cohort of only those series_ids 

In [ ]:
# Form the cohort_df of only these series_ids 
cohort_df = cohort_df_all[cohort_df_all['SeriesInstanceUID'].isin(series_id_list)]
cohort_df 

,SeriesInstanceUID,PatientID,StudyInstanceUID,Modality,num_instances,num_differences,max_difference,min_difference,idc_url,gcs_url
3,1.2.840.113654.2.55.12134616178609872830868013...,109589,1.2.840.113654.2.55.29851631757798774022898277...,CT,103,1,3.000000000,3.000000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/3b437734-da79-4150-a0...
5,1.2.840.113654.2.55.12134616178609872830868013...,109589,1.2.840.113654.2.55.29851631757798774022898277...,CT,103,1,3.000000000,3.000000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/c2201ee4-6379-4cd3-93...
17,1.2.840.113654.2.55.12134616178609872830868013...,109589,1.2.840.113654.2.55.29851631757798774022898277...,CT,103,1,3.000000000,3.000000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/8be8096f-703d-4451-8b...
18,1.2.840.113654.2.55.12134616178609872830868013...,109589,1.2.840.113654.2.55.29851631757798774022898277...,CT,103,1,3.000000000,3.000000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/06c51acb-1b95-4fff-a3...
33,1.2.840.113654.2.55.12134616178609872830868013...,109589,1.2.840.113654.2.55.29851631757798774022898277...,CT,103,1,3.000000000,3.000000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/ede4c7c0-1bc8-4221-aa...
...,...,...,...,...,...,...,...,...,...,...
169696,1.2.840.113654.2.55.13672741274082781122803247...,115020,1.2.840.113654.2.55.27598655255944753733836688...,CT,141,1,2.500000000,2.500000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/380434fe-388b-4485-82...
169699,1.2.840.113654.2.55.12639299486083929703391531...,114798,1.2.840.113654.2.55.99714127297866853721436798...,CT,119,1,2.500000000,2.500000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/fc813b50-f060-4db8-a3...
169712,1.2.840.113654.2.55.14135918910603311597323469...,119129,1.2.840.113654.2.55.27467837192329084551179783...,CT,129,1,2.500000000,2.500000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/95995de9-4202-4c78-b5...
169716,1.2.840.113654.2.55.12278645495544285873262409...,119496,1.2.840.113654.2.55.14372558290700463198816906...,CT,124,1,2.500000000,2.500000000,https://viewer.imaging.datacommons.cancer.gov/...,gs://public-datasets-idc/249b9208-3546-423a-9c...


In [ ]:
len(list(set(cohort_df['SeriesInstanceUID'].values)))

50

Create the view if it doesn't already exist. 

In [ ]:
# if (gce_vm):
#   client = bigquery.Client()
# else: 
#   client = bigquery.Client(project=project_name)

# view_id = '.'.join([project_name, dataset_table_id, table_view_id_name])
# print (view_id)

# view = bigquery.Table(view_id)
# view.view_query = query

# try:
#   view = client.create_table(view)
#   view_exists = 0 
# except:
#   print ('view ' + str(view) + ' already exists')
#   view_exists = 1 
# print ('view_exists: ' + str(view_exists))

idc-external-018.dataset_nlst.nlst_pick_single_series_per_study_cancer_positive_filtering_missing_slices_axial
view idc-external-018.dataset_nlst.nlst_pick_single_series_per_study_cancer_positive_filtering_missing_slices_axial already exists
view_exists: 1
